In [7]:
#cell 1
import os
import glob
import logging
import math
import datetime
from datetime import date, timedelta
import shutil
import time
import base64
import io
import random
import numpy as np

import pandas as pd
import dash
from dash import dcc, html, Input, Output, State, callback_context, dash_table
import gunicorn
import plotly.express as px

# Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ----------------- Config -----------------
# Set to dummy mode
USE_DUMMY_DATA = True

# Paths (dummy - won't actually load files)
TARGETS_PATH   = "COC_TARGETS.csv"
ADOPTIONS_PATH = "2025_Adoptions2.csv"
LOANS_PATH     = "Trainer_Loans.csv"
ASSISTANT_PATH = "Ass_COC.csv"
AUDIT_PATH     = "adoption_changes_audit.csv"

# Persistence mode - set to csv for dummy data
PERSISTENCE = 'csv'

# UI styles (unchanged)
nav_btn     = {'padding':'8px 16px','fontSize':'14px','backgroundColor':'#003366','color':'white','border':'none','borderRadius':'4px'}
nav_btn_alt = {'padding':'8px 16px','fontSize':'14px','backgroundColor':'#006400','color':'white','border':'none','borderRadius':'4px'}
nav_links   = {'position':'absolute','top':'20px','right':'20px','display':'flex'}
lbl         = {'fontWeight':'bold'}
grid5       = {'display':'grid','gridTemplateColumns':'repeat(7,1fr)','gap':'10px','padding':'10px','backgroundColor':'#b0e0e6','borderRadius':'10px','margin':'10px'}
grid6       = {'display':'grid','gridTemplateColumns':'repeat(6,1fr)','gap':'20px','padding':'20px','backgroundColor':'#f4f7fa','borderRadius':'10px','marginBottom':'20px'}
grid_cmp    = {'display':'grid','gridTemplateColumns':'repeat(5,1fr)','gap':'10px','padding':'10px','backgroundColor':'#b0e0e6','borderRadius':'10px','margin':'10px'}
submit_btn  = {'padding':'10px 20px','fontSize':'16px','backgroundColor':'#003366','color':'white','border':'none','borderRadius':'4px'}

# ----------------- Dummy Data Generation -----------------

def generate_dummy_targets():
    """Generate dummy targets data"""
    months = ['January', 'February', 'March', 'April', 'May', 'June', 
              'July', 'August', 'September', 'October', 'November', 'December']
    counties = ['Nairobi', 'Mombasa', 'Kisumu', 'Nakuru', 'Eldoret', 'Kakamega', 'Kisii', 'Kericho']
    clients = ['Sistema Bio', 'Bimeda', 'Elanco', 'Ceva', 'MSD', 'Zoetis']
    coordinators = ['John Doe', 'Jane Smith', 'Robert Johnson', 'Mary Williams', 'David Brown']
    categories = ['Dairy', 'Poultry', 'Crop', 'Veterinary', 'Equipment']
    
    data = []
    for month in months:
        for county in counties:
            for client in clients:
                for coordinator in coordinators[:3]:  # Limit coordinators per combination
                    for category in random.sample(categories, 2):  # 2 categories per combination
                        data.append({
                            'Month': month,
                            'County': county,
                            'Client': client,
                            'Co-ordinator': coordinator,
                            'Category': category,
                            'Year': '2025',
                            'Cumulative Target Per Month': random.randint(50000, 200000),
                            'Daily Target': random.randint(2000, 8000)
                        })
    
    return pd.DataFrame(data)

def generate_dummy_adoptions():
    """Generate dummy adoption data"""
    counties = ['Nairobi', 'Mombasa', 'Kisumu', 'Nakuru', 'Eldoret', 'Kakamega', 'Kisii', 'Kericho']
    brands = ['Sistema Bio', 'Bimeda', 'Elanco', 'Ceva', 'MSD', 'Zoetis']
    categories = ['Dairy', 'Poultry', 'Crop', 'Veterinary', 'Equipment']
    products = ['Vaccine A', 'Vaccine B', 'Feed Supplement', 'Equipment X', 'Medicine Y', 'Tool Z']
    trainers = [f'Trainer {i}' for i in range(1, 21)]
    statuses = ['Verified', 'Pending', 'Rejected', 'Undigitized', 'Unverified']
    
    # Generate dates for the past 180 days
    end_date = datetime.date.today()
    start_date = end_date - timedelta(days=180)
    date_range = pd.date_range(start_date, end_date)
    
    data = []
    for i in range(500):  # Generate 500 records
        record_date = random.choice(date_range)
        county = random.choice(counties)
        brand = random.choice(brands)
        category = random.choice(categories)
        product = random.choice(products)
        trainer = random.choice(trainers)
        status = random.choice(statuses)
        
        amount = random.randint(1000, 50000)
        quantity = random.randint(1, 10) if random.random() > 0.3 else 0
        actual_sales = amount if random.random() > 0.2 else 0
        orders = amount if random.random() > 0.2 else 0
        
        data.append({
            'RowID': f'r{int(time.time())}_{i}',
            'Date': record_date,
            'County': county,
            'Brand': brand,
            'Category': category,
            'Product': product,
            'Trainer Name': trainer,
            'Amount': amount,
            'Quantity': quantity,
            'Actual Sales': actual_sales,
            'Orders': orders,
            'Status': status,
            'Rejection Reason': random.choice(['', 'Incomplete documentation', 'Wrong product', 'Duplicate entry', '']) if status == 'Rejected' else ''
        })
    
    df = pd.DataFrame(data)
    # Ensure proper types
    df['Date'] = pd.to_datetime(df['Date'])
    return df

def generate_dummy_loans():
    """Generate dummy loan data"""
    trainers = [f'Trainer {i}' for i in range(1, 21)]
    
    data = []
    for trainer in trainers:
        data.append({
            'Trainer Name': trainer,
            'Loan Borrowed': random.randint(0, 50000),
            'Loan Repaid': random.randint(0, 30000)
        })
    
    return pd.DataFrame(data)

def generate_dummy_assistant():
    """Generate dummy assistant COC data"""
    coordinators = ['Assistant A', 'Assistant B', 'Assistant C', 'Assistant D']
    counties = ['Nairobi', 'Mombasa', 'Kisumu', 'Nakuru']
    
    # Generate dates for the past 90 days
    end_date = datetime.date.today()
    start_date = end_date - timedelta(days=90)
    date_range = pd.date_range(start_date, end_date, freq='D')
    
    data = []
    for date in date_range:
        for coordinator in coordinators:
            county = random.choice(counties)
            achieved = random.randint(10000, 80000)
            target = achieved + random.randint(-20000, 20000)
            
            data.append({
                'Date': date,
                'Co-ordinator': coordinator,
                'County': county,
                'Achieved': achieved,
                'Target': max(target, 10000)  # Ensure target is positive
            })
    
    df = pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'])
    return df

def generate_dummy_ndume():
    """Generate dummy Ndume adoption data"""
    counties = ['Bomet', 'Bungoma', 'Nandi', 'Nyamira', 'Taita Taveta', 'Trans Nzoia']
    bull_names = ['Delta Fast Lane', 'Mandas Upside', 'Delta Starliner', 'Other Bull']
    vet_names = [f'Vet {i}' for i in range(1, 11)]
    
    data = []
    for _ in range(300):  # Generate 300 records
        county = random.choice(counties)
        bull = random.choice(bull_names)
        vet = random.choice(vet_names)
        
        # Generate a date in 2025
        date = datetime.date(2025, random.randint(5, 12), random.randint(1, 28))
        
        data.append({
            'vet_name': vet,
            'County': county,
            'bull_name': bull,
            'Number_of_Inseminations': random.randint(1, 5),
            'date': date
        })
    
    df = pd.DataFrame(data)
    return df

def generate_dummy_vaccines():
    """Generate dummy vaccine data"""
    counties = ['Nyandarua', 'Nyeri', 'Laikipia', 'Vihiga', 'Nandi', 'Tharaka-Nithi', 'Nyamira']
    treatments = ['ECF Vaccine', 'FMD Vaccine', 'LSD Vaccine', 'BQ/Antharax', 'CCPP', 'RVF', 'PPR']
    
    data = []
    for county in counties:
        for treatment in treatments:
            data.append({
                'County': county,
                'Treatment': treatment,
                'Number_of_Vaccines': random.randint(50, 500)
            })
    
    return pd.DataFrame(data)

def generate_dummy_farmer_reach():
    """Generate dummy farmer reach data"""
    counties = ['Nyandarua', 'Nyeri', 'Laikipia', 'Vihiga', 'Nandi', 'Tharaka-Nithi', 'Nyamira']
    companies = ['Company A', 'Company B', 'Company C']
    topics = ['Dairy Farming', 'Poultry Management', 'Crop Rotation', 'Veterinary Care']
    
    data = []
    for county in counties:
        for company in companies:
            for topic in topics:
                data.append({
                    'company_name': company,
                    'Topic_name': topic,
                    'County': county,
                    'farmer_reach': random.randint(100, 1000)
                })
    
    return pd.DataFrame(data)

# ----------------- Lightweight caching -----------------
_CACHE = {}

def get_today():
    return datetime.date.today()

def _find_latest_file(base_path_or_pattern):
    """Dummy implementation - always returns None"""
    return None

def _load_if_new(key, path_or_pattern, loader_func):
    if USE_DUMMY_DATA:
        # Return from cache or generate dummy data
        cached = _CACHE.get(key)
        if cached:
            return cached['df']
        else:
            # Generate dummy data based on key
            if key == 'targets':
                df = generate_dummy_targets()
            elif key == 'df1':
                df = generate_dummy_adoptions()
            elif key == 'loans':
                df = generate_dummy_loans()
            elif key == 'adoptions_perf':
                df = generate_dummy_adoptions()  # Same as df1 for now
            elif key == 'assistant':
                df = generate_dummy_assistant()
            else:
                df = pd.DataFrame()
            
            _CACHE[key] = {'mtime': time.time(), 'df': df, 'path': None}
            return df
    else:
        # Original implementation
        latest = _find_latest_file(path_or_pattern)
        if latest is None:
            cached = _CACHE.get(key)
            return cached['df'] if cached else pd.DataFrame()
        try:
            mtime = os.path.getmtime(latest)
        except Exception:
            mtime = None
        cached = _CACHE.get(key)
        if cached and cached.get('mtime') == mtime:
            return cached['df']
        df = loader_func(latest)
        _CACHE[key] = {'mtime': mtime, 'df': df, 'path': latest}
        logging.info(f"Loaded {latest} into cache key={key}")
        return df

# ----------------- Enhanced Date Parser -----------------
def parse_date_flexible(date_str):
    """
    Flexible date parser that handles multiple formats
    """
    if pd.isna(date_str) or date_str == '':
        return pd.NaT
    
    date_str = str(date_str).strip()
    
    formats = [
        '%d-%m-%y', '%d/%m/%y',
        '%d-%m-%Y', '%d/%m/%Y',  
        '%Y-%m-%d', '%Y/%m/%d',
        '%m-%d-%y', '%m/%d/%y',
        '%m-%d-%Y', '%m/%d/%Y',
    ]
    
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt, errors='raise')
        except (ValueError, TypeError):
            continue
    
    try:
        return pd.to_datetime(date_str, dayfirst=True, errors='coerce')
    except:
        return pd.NaT

# ----------------- File-specific loaders -----------------
def _load_targets(path):
    if USE_DUMMY_DATA:
        return generate_dummy_targets()
    # Original implementation would go here...

def _load_df1(path):
    if USE_DUMMY_DATA:
        return generate_dummy_adoptions()
    # Original implementation would go here...

def _load_loans(path):
    if USE_DUMMY_DATA:
        return generate_dummy_loans()
    # Original implementation would go here...

def _load_adoptions_perf(path):
    if USE_DUMMY_DATA:
        return generate_dummy_adoptions()  # Same as df1
    # Original implementation would go here...

def _load_assistant(path):
    if USE_DUMMY_DATA:
        return generate_dummy_assistant()
    # Original implementation would go here...

# Getter functions
def get_targets_df():
    return _load_if_new('targets', TARGETS_PATH, _load_targets)

def get_df1():
    return _load_if_new('df1', ADOPTIONS_PATH, _load_df1)

def get_df_loans():
    return _load_if_new('loans', LOANS_PATH, _load_loans)

def get_adoptions_df():
    return _load_if_new('adoptions_perf', ADOPTIONS_PATH, _load_adoptions_perf)

def get_assistant_df():
    return _load_if_new('assistant', ASSISTANT_PATH, _load_assistant)

# ----------------- DB fetch functions (dummy versions) -----------------
def fetch_active_trainers(start_date, end_date):
    if USE_DUMMY_DATA:
        counties = ['Nyandarua', 'Nyeri', 'Laikipia', 'Vihiga', 'Nandi', 'Tharaka-Nithi', 'Nyamira']
        
        # Generate date range
        start = pd.to_datetime(start_date)
        end = pd.to_datetime(end_date)
        date_range = pd.date_range(start, end)
        
        data = []
        for date in date_range:
            if random.random() > 0.7:  # 70% chance of no data for a given day
                for _ in range(random.randint(1, 10)):
                    data.append({
                        'Date': date,
                        'County': random.choice(counties),
                        'TrainerID': f'T{random.randint(1000, 9999)}'
                    })
        
        df = pd.DataFrame(data) if data else pd.DataFrame(columns=['Date', 'County', 'TrainerID'])
        df['Date'] = pd.to_datetime(df['Date']).dt.normalize()
        df['County'] = df['County'].astype(str).str.title().str.strip()
        return df
    else:
        # Original database query would go here...
        pass

def fetch_vaccines(start_date, end_date):
    if USE_DUMMY_DATA:
        df = generate_dummy_vaccines()
        df['County'] = df['County'].astype(str).str.title().str.strip()
        return df
    else:
        # Original database query would go here...
        pass

def fetch_farmer_reach(start_date, end_date):
    if USE_DUMMY_DATA:
        df = generate_dummy_farmer_reach()
        df['County'] = df['County'].astype(str).str.title().str.strip()
        df['Topic_name'] = df['Topic_name'].astype(str).str.strip()
        return df
    else:
        # Original database query would go here...
        pass

# ----------------- Ndume plotting (dummy version) -----------------
# Generate dummy data for Ndume plots
df_nd = generate_dummy_ndume()

def normalize_bull(name):
    n = str(name).strip().lower()
    if "fast" in n and "lane" in n:
        return "Delta Fast Lane"
    if "upside" in n:
        return "Mandas Upside"
    if "starliner" in n:
        return "Delta Starliner"
    return str(name).strip().title()

df_nd['bull_name'] = df_nd['bull_name'].apply(normalize_bull)
fig_nd1 = px.bar(df_nd.groupby('bull_name').size().reset_index(name='count'), 
                 x='bull_name', y='count', 
                 title='Number of Inseminations per Bull Name (Dummy Data)', 
                 text_auto=True, template='plotly_white')
fig_nd1.update_layout(xaxis_tickangle=-45)

fig_nd2 = px.bar(df_nd.groupby('County').size().reset_index(name='count'), 
                 x='County', y='count', 
                 title='Number of Inseminations per County (Dummy Data)', 
                 text_auto=True, template='plotly_white')
fig_nd2.update_layout(xaxis_tickangle=-45)

fig_nd3 = px.histogram(df_nd, x='County', color='bull_name', barmode='group', 
                       title='Bull Inseminations per County by Bull Name (Dummy Data)', 
                       template='plotly_white')
fig_nd3.update_layout(xaxis_tickangle=-45)

# ----------------- Persistence helpers -----------------
def _backup_file(path):
    # Dummy implementation for portfolio
    return None

def persist_adoptions_csv(df, path=ADOPTIONS_PATH):
    try:
        # In dummy mode, just update the cache
        if USE_DUMMY_DATA:
            _CACHE['df1'] = {'mtime': time.time(), 'df': df, 'path': None}
            _CACHE['adoptions_perf'] = {'mtime': time.time(), 'df': df, 'path': None}
            logger.info('Persisted adoptions to cache (dummy mode)')
            return True, 'Saved to cache'
        else:
            # Original implementation would go here...
            pass
    except Exception as e:
        logger.exception('Failed to persist adoptions')
        return False, str(e)

def persist_adoptions_mysql(df, table_name='adoptions'):
    try:
        # Dummy implementation for portfolio
        logger.info('Persisted adoptions (dummy MySQL mode)')
        return True, 'Saved to dummy MySQL'
    except Exception as e:
        logger.exception('Failed to persist to MySQL')
        return False, str(e)

def persist_adoptions(df):
    if PERSISTENCE == 'csv':
        return persist_adoptions_csv(df)
    else:
        return persist_adoptions_mysql(df)

def append_audit_entry(action, rowid, col_changed, old, new, user):
    try:
        # Dummy implementation - just log to console
        logger.info(f'Audit: {action} {rowid} {col_changed} {old} -> {new} by {user}')
    except Exception as e:
        logger.exception(f'Failed to write audit entry: {e}')

# NEW: Enhanced file parser for both CSV and Excel
def parse_uploaded_file(contents, filename):
    """
    Enhanced file parser that handles both CSV and Excel files
    """
    try:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        
        if USE_DUMMY_DATA:
            # Generate dummy data instead of parsing
            df = generate_dummy_adoptions()
            # Add a note that this is uploaded data
            df['Upload_Source'] = filename
            return True, 'Parsed successfully (dummy data generated)', df
        else:
            # Original implementation would go here...
            pass
    except Exception as e:
        logger.exception('Failed to parse uploaded file')
        return False, str(e), None

# OLD: Keep for backward compatibility
def parse_uploaded_csv(contents):
    """Deprecated - use parse_uploaded_file instead"""
    return parse_uploaded_file(contents, 'upload.csv')

# CELL 2/4
# Layouts with Role-Based Admin Panel

app = dash.Dash(__name__, suppress_callback_exceptions=True, external_stylesheets=[
    'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css'
])
ADOPTION_ID, PERFORMANCE_ID, NDUME_ID, COMPARISON_ID, SELF_SERVICE_ID = "adoption","performance","ndume","comparison","self-service"
server = app.server
app.title = "ADOPTION TRACKER - PORTFOLIO DEMO (Dummy Data)"

# Role-based authentication credentials
USER_CREDENTIALS = {
    "admin": {"password": "admin123", "role": "admin"},
    "qa": {"password": "qa123", "role": "qa"}
}

# Additional UI styles
login_style = {
    'padding': '40px', 
    'maxWidth': '400px', 
    'margin': '100px auto', 
    'backgroundColor': 'white', 
    'borderRadius': '15px',
    'boxShadow': '0 4px 8px rgba(0,0,0,0.1)'
}

# Helper to create editable DataTable for QA role
def make_qa_table_component():
    df = get_df1().copy()
    # Add note about dummy data
    if 'Note' not in df.columns:
        df['Note'] = 'Dummy Data for Portfolio Demo'
    
    # Ensure RowID exists
    if 'RowID' not in df.columns:
        df.insert(0, 'RowID', [f"r{int(time.time())}_{i}" for i in range(len(df))])
    
    # Add Rejection Reason column if not exists
    if 'Rejection Reason' not in df.columns:
        df['Rejection Reason'] = ''
    
    # Date formatting
    if 'Date' in df.columns:
        df['Date'] = df['Date'].apply(
            lambda x: x.strftime('%d-%m-%Y') if pd.notna(x) else 'Invalid Date'
        )
    
    cols = list(df.columns)
    columns = []
    for c in cols:
        col_def = {"name": c, "id": c}
        if c == 'RowID':
            col_def["editable"] = False
            col_def["hideable"] = True
        elif c == 'Status':
            col_def["editable"] = True
            col_def['presentation'] = 'dropdown'
        elif c == 'Rejection Reason':
            col_def["editable"] = True
            col_def["name"] = "Rejection Reason/Comments"
        elif c == 'Date':
            col_def["editable"] = True
        elif c == 'Note':
            col_def["editable"] = False
        else:
            col_def["editable"] = False
            
        columns.append(col_def)
    
    table = dash_table.DataTable(
        id='qa-table',
        columns=columns,
        data=df.fillna("").to_dict('records'),
        editable=True,
        row_selectable='multi',
        selected_rows=[],
        page_action='native',
        page_size=15,
        dropdown={
            'Status': {
                'options': [
                    {"label":"✅ Verified","value":"Verified"},
                    {"label":"⏳ Pending","value":"Pending"},
                    {"label":"❌ Rejected","value":"Rejected"},
                    {"label":"📄 Undigitized","value":"Undigitized"},
                    {"label":"❓ Unverified","value":"Unverified"}
                ]
            }
        },
        style_table={'overflowX': 'auto', 'borderRadius': '8px'},
        style_cell={
            'minWidth':'120px', 'width':'150px', 'maxWidth':'250px', 
            'whiteSpace':'normal', 'textAlign': 'left',
            'fontFamily': 'Arial, sans-serif'
        },
        style_header={
            'backgroundColor': '#27ae60',
            'color': 'white',
            'fontWeight': 'bold'
        },
        style_data_conditional=[
            {
                'if': {
                    'filter_query': '{Status} = "Verified"',
                },
                'backgroundColor': '#d4edda',
                'color': '#155724'
            },
            {
                'if': {
                    'filter_query': '{Status} = "Rejected"',
                },
                'backgroundColor': '#f8d7da',
                'color': '#721c24'
            },
            {
                'if': {
                    'filter_query': '{Status} = "Pending"',
                },
                'backgroundColor': '#fff3cd',
                'color': '#856404'
            },
            {
                'if': {
                    'filter_query': '{Note} = "Dummy Data for Portfolio Demo"',
                },
                'backgroundColor': '#e3f2fd',
                'color': '#1565c0',
                'fontStyle': 'italic'
            },
            {
                'if': {'state': 'selected'},
                'backgroundColor': 'rgba(39, 174, 96, 0.2)',
                'border': '1px solid #27ae60'
            }
        ],
        tooltip_data=[
            {
                column: {'value': str(value), 'type': 'markdown'}
                for column, value in row.items()
            } for row in df.to_dict('records')
        ],
        tooltip_duration=None
    )
    return table
    
# Helper to create editable DataTable for Admin role
def make_admin_table_component():
    df = get_df1().copy()
    # Add note about dummy data
    if 'Note' not in df.columns:
        df['Note'] = 'Dummy Data for Portfolio Demo'
    
    # Ensure RowID exists
    if 'RowID' not in df.columns:
        df.insert(0, 'RowID', [f"r{int(time.time())}_{i}" for i in range(len(df))])
    
    # Add Rejection Reason column if not exists
    if 'Rejection Reason' not in df.columns:
        df['Rejection Reason'] = ''
    
    # Date formatting
    if 'Date' in df.columns:
        df['Date'] = df['Date'].apply(
            lambda x: x.strftime('%d-%m-%Y') if pd.notna(x) else 'Invalid Date'
        )
    
    cols = list(df.columns)
    columns = []
    for c in cols:
        col_def = {"name": c, "id": c}
        if c == 'RowID':
            col_def["editable"] = False
            col_def["hideable"] = True
        elif c == 'Status':
            col_def["editable"] = True
            col_def['presentation'] = 'dropdown'
        elif c == 'Date':
            col_def["editable"] = True
        elif c == 'Note':
            col_def["editable"] = False
        else:
            col_def["editable"] = True
            
        columns.append(col_def)
    
    table = dash_table.DataTable(
        id='admin-table',
        columns=columns,
        data=df.fillna("").to_dict('records'),
        editable=True,
        row_selectable='multi',
        selected_rows=[],
        page_action='native',
        page_size=15,
        dropdown={
            'Status': {
                'options': [
                    {"label":"✅ Verified","value":"Verified"},
                    {"label":"⏳ Pending","value":"Pending"},
                    {"label":"❌ Rejected","value":"Rejected"},
                    {"label":"📄 Undigitized","value":"Undigitized"},
                    {"label":"❓ Unverified","value":"Unverified"}
                ]
            }
        },
        style_table={'overflowX': 'auto', 'borderRadius': '8px'},
        style_cell={
            'minWidth':'120px', 'width':'150px', 'maxWidth':'250px', 
            'whiteSpace':'normal', 'textAlign': 'left',
            'fontFamily': 'Arial, sans-serif'
        },
        style_header={
            'backgroundColor': '#8e44ad',
            'color': 'white',
            'fontWeight': 'bold'
        },
        style_data_conditional=[
            {
                'if': {
                    'filter_query': '{Status} = "Verified"',
                },
                'backgroundColor': '#d4edda',
                'color': '#155724'
            },
            {
                'if': {
                    'filter_query': '{Status} = "Rejected"',
                },
                'backgroundColor': '#f8d7da',
                'color': '#721c24'
            },
            {
                'if': {
                    'filter_query': '{Status} = "Pending"',
                },
                'backgroundColor': '#fff3cd',
                'color': '#856404'
            },
            {
                'if': {
                    'filter_query': '{Note} = "Dummy Data for Portfolio Demo"',
                },
                'backgroundColor': '#e3f2fd',
                'color': '#1565c0',
                'fontStyle': 'italic'
            },
            {
                'if': {'state': 'selected'},
                'backgroundColor': 'rgba(142, 68, 173, 0.2)',
                'border': '1px solid #8e44ad'
            }
        ],
        tooltip_data=[
            {
                column: {'value': str(value), 'type': 'markdown'}
                for column, value in row.items()
            } for row in df.to_dict('records')
        ],
        tooltip_duration=None
    )
    return table

# Login layout
def layout_login():
    return html.Div([
        html.Div([
            html.H1("ADOPTION TRACKER - PORTFOLIO DEMO", style={'textAlign':'center','color':'#2c3e50','marginBottom':'10px'}),
            html.H2("Admin/QA Login", style={'textAlign':'center','color':'#003366','marginBottom':'30px'}),
            html.P("This is a portfolio demo with dummy data. Use admin/admin123 or qa/qa123 to login.", 
                  style={'textAlign':'center','color':'#666','marginBottom':'20px','fontStyle':'italic'}),
            html.Div([
                html.Label("Username:", style=lbl),
                dcc.Input(
                    id='login-username', 
                    type='text', 
                    value='admin',  # Pre-fill for demo
                    style={'width': '100%', 'padding': '10px', 'marginBottom': '20px', 'borderRadius': '5px', 'border': '1px solid #ddd'}
                )
            ]),
            html.Div([
                html.Label("Password:", style=lbl),
                dcc.Input(
                    id='login-password', 
                    type='password', 
                    value='admin123',  # Pre-fill for demo
                    style={'width': '100%', 'padding': '10px', 'marginBottom': '30px', 'borderRadius': '5px', 'border': '1px solid #ddd'}
                )
            ]),
            html.Button("Login", id='login-button', style={**submit_btn, 'width': '100%'}),
            html.Div(id='login-message', style={'marginTop': '15px', 'textAlign': 'center'}),
            html.Hr(style={'margin': '30px 0'}),
            html.Div([
                dcc.Link("← Back to Main Dashboard", href=f"/{ADOPTION_ID}", style={'color': '#003366', 'textDecoration': 'none'})
            ], style={'textAlign': 'center'})
        ], style=login_style)
    ], style={'backgroundColor': '#b0e0e6', 'minHeight': '100vh', 'padding': '20px'})

# Admin layout
def layout_admin():
    df1 = get_df1()
    return html.Div([
        html.Div([
            html.H1("ADMIN PANEL - PORTFOLIO DEMO", style={'textAlign':'center','color':'#8e44ad','padding':'20px','backgroundColor':'#f4e6ff'}),
            html.Div([
                dcc.Link(html.Button("← Back to Dashboard", style=nav_btn), href=f"/{ADOPTION_ID}"),
                html.Button("Logout", id='logout-button', style={**nav_btn, 'backgroundColor':'#c0392b', 'marginLeft':'10px'})
            ], style=nav_links)
        ], style={'position':'relative', 'backgroundColor': 'white', 'borderRadius': '10px', 'marginBottom': '20px'}),
        
        # Demo Notice
        html.Div([
            html.H3("📊 Portfolio Demo Notice", style={'textAlign': 'center', 'color': '#8e44ad'}),
            html.P("This is a portfolio demonstration using synthetic data. All data is randomly generated for showcase purposes.", 
                  style={'textAlign': 'center', 'color': '#666', 'padding': '10px 20px'})
        ], style={'backgroundColor': '#e3f2fd', 'padding': '15px', 'margin': '20px', 'borderRadius': '10px', 'border': '2px solid #bbdefb'}),
        
        # File Management Section
        html.Div([
            html.H2("File Management", style={'textAlign': 'center', 'marginBottom': '20px', 'color': '#8e44ad'}),
            html.Div([
                dcc.Upload(
                    id='admin-upload-data', 
                    children=html.Div([
                        html.I(className="fas fa-cloud-upload-alt", style={'fontSize': '24px', 'marginBottom': '10px'}),
                        html.Br(),
                        'Drag and Drop or ',
                        html.A('Select a CSV/Excel File', style={'color': '#8e44ad', 'fontWeight': 'bold'})
                    ]), 
                    style={
                        'width':'100%','height':'120px','lineHeight':'30px',
                        'borderWidth':'2px','borderStyle':'dashed','borderRadius':'8px',
                        'textAlign':'center','marginTop':'12px', 'borderColor': '#8e44ad',
                        'backgroundColor': '#f8f6ff', 'padding': '20px'
                    }, 
                    multiple=False
                ),
                html.Div([
                    html.Label('Upload mode:', style=lbl), 
                    dcc.RadioItems(
                        id='admin-upload-mode', 
                        options=[
                            {'label':' 📥 Append to current data','value':'append'},
                            {'label':' 🔄 Replace entire table','value':'replace'}
                        ], 
                        value='append',
                        labelStyle={'display': 'block', 'margin': '8px 0', 'fontSize': '14px'}
                    )
                ], style={'marginTop': '20px'}),
                html.Button('🗑️ Delete All Data', id='admin-delete-data', n_clicks=0,
                           style={**nav_btn, 'backgroundColor': '#e74c3c', 'marginTop': '15px', 'width': '200px'})
            ], style={'padding':'20px','backgroundColor':'#f8f9fa','borderRadius':'10px','marginBottom':'20px'})
        ]),

        # Upload Status Display
        html.Div([
            html.H3("Upload Status", style={'textAlign': 'center', 'marginBottom': '15px', 'color': '#8e44ad'}),
            html.Div(id='admin-upload-status', children=[
                html.Div([
                    html.I(className="fas fa-info-circle", style={'marginRight': '10px'}),
                    "No file uploaded yet. Upload will generate dummy data for demo purposes."
                ], style={
                    'padding': '15px', 
                    'backgroundColor': '#e3f2fd', 
                    'borderRadius': '8px',
                    'border': '1px solid #bbdefb',
                    'textAlign': 'center'
                })
            ])
        ], style={'padding': '15px', 'margin': '20px', 'backgroundColor': 'white', 'borderRadius': '10px', 'border': '2px solid #e3f2fd'}),

        # Action Messages
        html.Div(id='admin-action-message', style={'textAlign':'center','color':'#2c3e50','marginTop':'8px', 'fontWeight': 'bold', 'minHeight': '30px'}),

        # Data Table with Full Edit Access
        html.Div([
            html.H2("Data Management", style={'textAlign': 'center', 'marginBottom': '20px', 'color': '#8e44ad'}),
            html.Div(id='admin-table-container', children=[make_admin_table_component()], 
                    style={'margin': '20px', 'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '10px'})
        ]),

        # Audit Logs Section
        html.Div([
            html.H2("Audit Logs", style={'textAlign': 'center', 'marginBottom': '15px', 'color': '#8e44ad'}),
            html.Div(id='admin-audit-preview', 
                    style={
                        'maxHeight':'400px','overflowY':'auto','border':'1px solid #eee',
                        'padding':'15px','background':'#fff','marginTop':'12px',
                        'borderRadius': '5px', 'fontFamily': 'monospace', 'fontSize': '12px'
                    })
        ], style={'padding': '20px', 'margin': '20px', 'backgroundColor': '#f8f9fa', 'borderRadius': '10px'})

    ], style={'backgroundColor':'#e9ecef','minHeight':'100vh','padding':'0 20px 20px 20px'})

# QA Person Layout
def layout_qa():
    df1 = get_df1()
    return html.Div([
        html.Div([
            html.H1("QA PANEL - PORTFOLIO DEMO", style={'textAlign':'center','color':'#27ae60','padding':'20px','backgroundColor':'#e6ffe6'}),
            html.Div([
                dcc.Link(html.Button("← Back to Dashboard", style=nav_btn), href=f"/{ADOPTION_ID}"),
                html.Button("Logout", id='logout-button', style={**nav_btn, 'backgroundColor':'#c0392b', 'marginLeft':'10px'})
            ], style=nav_links)
        ], style={'position':'relative', 'backgroundColor': 'white', 'borderRadius': '10px', 'marginBottom': '20px'}),
        
        # Demo Notice
        html.Div([
            html.H3("🔍 QA Panel - Portfolio Demo", style={'textAlign': 'center', 'color': '#27ae60'}),
            html.P("This QA panel demonstrates data validation workflows using synthetic data. All records are randomly generated.", 
                  style={'textAlign': 'center', 'color': '#666', 'padding': '10px 20px'})
        ], style={'backgroundColor': '#e3f2fd', 'padding': '15px', 'margin': '20px', 'borderRadius': '10px', 'border': '2px solid #bbdefb'}),
        
        # QA Progress Dashboard
        html.Div([
            html.H2("QA Progress Dashboard", style={'textAlign': 'center', 'marginBottom': '20px', 'color': '#27ae60'}),
            html.Div(id='qa-progress-cards', style={'display': 'flex', 'justifyContent': 'center', 'flexWrap': 'wrap', 'gap': '20px', 'marginBottom': '20px'})
        ]),

        # Enhanced Filters with QA Theme
        html.Div([
            html.H3("Data Filters", style={'textAlign': 'center', 'marginBottom': '15px', 'color': '#27ae60'}),
            html.Div([
                html.Div([
                    html.Label("Date Range", style={**lbl, 'color': '#27ae60'}),
                    html.Div([
                        dcc.DatePickerSingle(
                            id="qa-start-date", 
                            date=get_today()-timedelta(days=30),
                            display_format='DD-MM-YYYY',
                            style={'marginRight': '10px'}
                        ),
                        dcc.DatePickerSingle(
                            id="qa-end-date", 
                            date=get_today(),
                            display_format='DD-MM-YYYY'
                        )
                    ], style={'display': 'flex', 'justifyContent': 'space-between'})
                ], style={'gridColumn': 'span 2'}),
                
                html.Div([
                    html.Label("County", style={**lbl, 'color': '#27ae60'}),
                    dcc.Dropdown(
                        id="qa-county-filter", 
                        options=[{'label':c,'value':c} for c in sorted(df1['County'].dropna().unique())], 
                        multi=True,
                        placeholder="Select counties..."
                    )
                ]),
                
                html.Div([
                    html.Label("Brand", style={**lbl, 'color': '#27ae60'}),
                    dcc.Dropdown(
                        id="qa-brand-filter", 
                        options=[{'label':c,'value':c} for c in sorted(df1['Brand'].dropna().unique())], 
                        multi=True,
                        placeholder="Select brands..."
                    )
                ]),
                
                html.Div([
                    html.Label("Category", style={**lbl, 'color': '#27ae60'}),
                    dcc.Dropdown(
                        id="qa-category-filter", 
                        options=[{'label':c,'value':c} for c in sorted(df1['Category'].dropna().unique())], 
                        multi=True,
                        placeholder="Select categories..."
                    )
                ]),
                
                html.Div([
                    html.Label("Trainer", style={**lbl, 'color': '#27ae60'}),
                    dcc.Dropdown(
                        id="qa-trainer-filter", 
                        options=[{'label':c,'value':c} for c in sorted(df1['Trainer Name'].dropna().unique())], 
                        multi=True,
                        placeholder="Select trainers..."
                    )
                ]),
                
                html.Div([
                    html.Label("Status", style={**lbl, 'color': '#27ae60'}),
                    dcc.Dropdown(
                        id="qa-status-filter", 
                        options=[{'label':c,'value':c} for c in sorted(df1['Status'].dropna().unique())], 
                        multi=True, 
                        value=['Pending', 'Unverified', 'Undigitized'],
                        placeholder="Select statuses..."
                    )
                ])
            ], style={
                'display': 'grid',
                'gridTemplateColumns': 'repeat(3, 1fr)',
                'gap': '15px',
                'padding': '20px'
            }),
            
            html.Div([
                html.Button("Apply Filters", id="qa-apply-filters", n_clicks=0, 
                           style={**submit_btn, 'backgroundColor': '#27ae60'}),
                html.Button("Clear Filters", id="qa-clear-filters", n_clicks=0,
                           style={**nav_btn, 'marginLeft': '10px'})
            ], style={'textAlign':'center','marginTop':'15px'})
        ], style={
            'padding': '20px', 
            'margin': '20px', 
            'backgroundColor': '#f0fff0', 
            'borderRadius': '10px',
            'border': '2px solid #27ae60'
        }),

        # QA Actions
        html.Div([
            html.H3("Bulk Actions", style={'textAlign': 'center', 'marginBottom': '15px', 'color': '#27ae60'}),
            html.Div([
                html.Button('✅ Bulk Approve Selected', id='qa-bulk-approve', n_clicks=0, 
                           style={**submit_btn, 'backgroundColor': '#27ae60', 'marginRight': '10px'}),
                html.Button('❌ Bulk Reject Selected', id='qa-bulk-reject', n_clicks=0, 
                           style={**submit_btn, 'backgroundColor':'#c0392b', 'marginRight': '10px'}),
                html.Button('💾 Save All Changes', id='qa-save-changes', n_clicks=0, 
                           style={**submit_btn, 'backgroundColor': '#f39c12', 'marginRight': '10px'}),
                html.Button('🔄 Refresh Data', id='qa-refresh', n_clicks=0, style=nav_btn)
            ], style={'textAlign':'center','padding':'20px','backgroundColor':'#f0f8f0','borderRadius':'10px','margin':'20px'})
        ]),

        html.Div(id='qa-action-message', style={'textAlign':'center','color':'#2c3e50','marginTop':'8px', 'fontWeight': 'bold', 'minHeight': '30px'}),
        
        # QA Data Table
        html.Div([
            html.H3("Records for Review", style={'textAlign': 'center', 'marginBottom': '15px', 'color': '#27ae60'}),
            html.Div([
                html.P("💡 This table shows synthetic data generated for portfolio demonstration.", 
                      style={'textAlign': 'center', 'color': '#666', 'fontStyle': 'italic', 'marginBottom': '10px'}),
                html.Div(id='qa-table-container', children=[make_qa_table_component()], 
                        style={'margin': '10px', 'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '10px'})
            ])
        ])

    ], style={'backgroundColor':'#f8fff8','minHeight':'100vh','padding':'0 20px 20px 20px'})

# Adoption layout (main dashboard)
def layout_adoption():
    df1 = get_df1()
    return html.Div([
        html.Div([
            html.H1("PRODUCT ADOPTIONS - PORTFOLIO DEMO", style={'textAlign':'center','color':'#2c3e50','padding':'20px'}),
            html.P("Dashboard demonstrating analytics capabilities with synthetic data", 
                  style={'textAlign':'center','color':'#666','marginBottom':'20px','fontStyle':'italic'}),
            html.Div([
                dcc.Link(html.Button("Performance Tracker", style=nav_btn), href=f"/{PERFORMANCE_ID}"),
                dcc.Link(html.Button("Ndume Adoptions", style={**nav_btn,'marginLeft':'10px'}), href=f"/{NDUME_ID}"),
                dcc.Link(html.Button("Comparison Analysis", style={**nav_btn_alt,'marginLeft':'10px'}), href=f"/{COMPARISON_ID}"),
                dcc.Link(html.Button("Admin/QA Panel", style={**nav_btn, 'backgroundColor':'#8e44ad','marginLeft':'10px'}), href=f"/{SELF_SERVICE_ID}")
            ], style=nav_links)
        ], style={'position':'relative'}),

        html.Div(style=grid5, children=[
            html.Div([html.Label("From Date", style=lbl), dcc.DatePickerSingle(
                id=f"{ADOPTION_ID}-start-date", 
                date=get_today()-timedelta(days=30), 
                display_format='DD-MM-YYYY'
            )]),
            html.Div([html.Label("To Date", style=lbl), dcc.DatePickerSingle(
                id=f"{ADOPTION_ID}-end-date", 
                date=get_today(), 
                display_format='DD-MM-YYYY'
            )]),
            html.Div([html.Label("County", style=lbl), dcc.Dropdown(id=f"{ADOPTION_ID}-county-filter", options=[{'label':c,'value':c} for c in sorted(df1['County'].dropna().unique())], multi=True, value=[])]),
            html.Div([html.Label("Brand", style=lbl), dcc.Dropdown(id=f"{ADOPTION_ID}-brand-filter", options=[{'label':c,'value':c} for c in sorted(df1['Brand'].dropna().unique())], multi=True, value=[])]),
            html.Div([html.Label("Category", style=lbl), dcc.Dropdown(id=f"{ADOPTION_ID}-category-filter", options=[{'label':c,'value':c} for c in sorted(df1['Category'].dropna().unique())], multi=True, value=[])]),
            html.Div([html.Label("Trainer", style=lbl), dcc.Dropdown(id=f"{ADOPTION_ID}-trainer-filter", options=[{'label':c,'value':c} for c in sorted(df1['Trainer Name'].dropna().unique())], multi=True, value=[])]),
            html.Div([html.Label("Status", style=lbl), dcc.Dropdown(id=f"{ADOPTION_ID}-status-filter", options=[{'label':c,'value':c} for c in sorted(df1['Status'].dropna().unique())], multi=True, value=[])]),
        ]),

        html.Div(html.Button("Submit", id=f"{ADOPTION_ID}-submit-button", n_clicks=0, style=submit_btn), style={'textAlign':'center','marginBottom':'20px'}),

        # Scorecards
        html.Div([
            html.Div([html.H3("Total Target (KES)", style={'color':'#2c3e50'}), html.H2(id=f"{ADOPTION_ID}-total-target", style={'color':'#e67e22','fontSize':'20px'})], style={'textAlign':'center','padding':'20px','background':'#f5f5f5','borderRadius':'15px','width':'280px','boxShadow':'2px 2px 10px rgba(0,0,0,0.2)','margin':'20px'}),
            html.Div([html.H3("Total Amount (KES)", style={'color':'#2c3e50'}), html.H2(id=f"{ADOPTION_ID}-total-amount", style={'color':'#27ae60','fontSize':'20px'})], style={'textAlign':'center','padding':'20px','background':'#f5f5f5','borderRadius':'15px','width':'280px','boxShadow':'2px 2px 10px rgba(0,0,0,0.2)','margin':'20px'}),
            html.Div([
                html.H3("Percentage Verified", style={'color':'#2c3e50'}),
                html.H2(id=f"{ADOPTION_ID}-verified-percentage", style={'color':'#27ae60','fontSize':'20px'}),
                html.H4(id=f"{ADOPTION_ID}-pending-percentage", style={'color':'#d35400','fontSize':'14px','marginTop':'6px'}),
                html.H4(id=f"{ADOPTION_ID}-rejected-percentage", style={'color':'#c0392b','fontSize':'14px','marginTop':'2px'}),
            ], style={'textAlign':'center','padding':'20px','background':'#f5f5f5','borderRadius':'15px','width':'360px','boxShadow':'2px 2px 10px rgba(0,0,0,0.2)','margin':'20px'}),
            html.Div([
                html.H3("Percentage Change", style={'color':'#2c3e50'}),
                html.H2(id=f"{ADOPTION_ID}-change-percentage", style={'color':'#2980b9','fontSize':'18px','marginTop':'8px'}),
                html.H4(id=f"{ADOPTION_ID}-timeline-note", style={'color':'#2980b9','fontSize':'12px','marginTop':'4px'}),
            ], style={'textAlign':'center','padding':'20px','background':'#f5f5f5','borderRadius':'15px','width':'320px','boxShadow':'2px 2px 10px rgba(0,0,0,0.2)','margin':'20px'}),
            html.Div(id=f"{ADOPTION_ID}-sistema-card", children=[html.H3("Sistema Bio — Installed Units", style={'color':'#2c3e50'}), html.H2(id=f"{ADOPTION_ID}-sistema-total-qty", style={'color':'#8e44ad','fontSize':'20px'})], style={'textAlign':'center','padding':'20px','background':'#f5f5f5','borderRadius':'12px','width':'320px','boxShadow':'2px 2px 10px rgba(0,0,0,0.2)','margin':'20px','display':'none'})
        ], style={'display':'flex','justifyContent':'center','flexWrap':'wrap','margin':'20px'}),

        # Standard charts
        html.Div([dcc.Graph(id=f"{ADOPTION_ID}-amount-per-brand", style={'width':'48%','padding':'10px'}), dcc.Graph(id=f"{ADOPTION_ID}-amount-per-category", style={'width':'48%','padding':'10px'})], style={'display':'flex','justifyContent':'center'}),
        html.Div([dcc.Graph(id=f"{ADOPTION_ID}-amount-per-product", style={'width':'48%','padding':'10px'}), dcc.Graph(id=f"{ADOPTION_ID}-amount-per-county", style={'width':'48%','padding':'10px'})], style={'display':'flex','justifyContent':'center'}),
        html.Div([dcc.Graph(id=f"{ADOPTION_ID}-amount-per-trainer", style={'width':'48%','padding':'10px'}), dcc.Graph(id=f"{ADOPTION_ID}-amount-per-day", style={'width':'48%','padding':'10px'})], style={'display':'flex','justifyContent':'center'}),

        # SISTEMA charts container
        html.Div(id=f"{ADOPTION_ID}-sistema-charts-container", children=[dcc.Graph(id=f"{ADOPTION_ID}-sistema-qty-per-product", style={'width':'48%','padding':'10px'}), dcc.Graph(id=f"{ADOPTION_ID}-sistema-installed-per-county", style={'width':'48%','padding':'10px'})], style={'display':'none','justifyContent':'center','marginTop':'10px'}),

        # Download button
        html.Div([
            html.Button("Download CSV", id="download-button", n_clicks=0, style=submit_btn),
            dcc.Download(id="download-dataframe-csv")
        ], style={"marginTop": "20px", "textAlign": "center"})

    ], style={'backgroundColor':'#b0e0e6','padding':'0 20px 20px 20px'})

# Performance layout
def layout_performance():
    adoptions_perf = get_adoptions_df()
    assistant_df = get_assistant_df()
    targets_df = get_targets_df()

    all_brands       = sorted(adoptions_perf["Brand"].dropna().unique()) if 'Brand' in adoptions_perf.columns else []
    all_categories   = sorted(adoptions_perf["Category"].dropna().unique()) if 'Category' in adoptions_perf.columns else []
    all_coordinators = sorted(targets_df["Co-ordinator"].dropna().unique()) if 'Co-ordinator' in targets_df.columns else []
    all_counties     = sorted(targets_df["County"].dropna().unique()) if 'County' in targets_df.columns else []

    return html.Div([
        html.Div([html.H1("TARGET/ACHIEVEMENT TRACKING - PORTFOLIO DEMO", style={'textAlign':'center','color':'#003366','padding':'20px'}),
                  html.P("Performance analytics with synthetic data", style={'textAlign':'center','color':'#666','marginBottom':'20px'}),
                  html.Div([dcc.Link(html.Button("Product Adoptions", style=nav_btn), href="/"),
                            dcc.Link(html.Button("Ndume Adoptions", style={**nav_btn,'marginLeft':'10px'}), href=f"/{NDUME_ID}"),
                            dcc.Link(html.Button("Comparison Analysis", style={**nav_btn_alt,'marginLeft':'10px'}), href=f"/{COMPARISON_ID}"),
                            dcc.Link(html.Button("Admin/QA Panel", style={**nav_btn, 'backgroundColor':'#8e44ad','marginLeft':'10px'}), href=f"/{SELF_SERVICE_ID}")], style=nav_links)], style={'position':'relative'}),
        html.Div(style=grid6, children=[
            html.Div([html.Label("From Date", style=lbl), dcc.DatePickerSingle(
                id=f"{PERFORMANCE_ID}-start-date", 
                date=get_today()-timedelta(days=30), 
                display_format='DD-MM-YYYY'
            )]),
            html.Div([html.Label("To Date", style=lbl), dcc.DatePickerSingle(
                id=f"{PERFORMANCE_ID}-end-date", 
                date=get_today(), 
                display_format='DD-MM-YYYY'
            )]),
            html.Div([html.Label("Co-ordinator", style=lbl), dcc.Dropdown(id=f"{PERFORMANCE_ID}-coordinator-filter", options=[{'label':c,'value':c} for c in all_coordinators], multi=True)]),
            html.Div([html.Label("County", style=lbl), dcc.Dropdown(id=f"{PERFORMANCE_ID}-county-filter", options=[{'label':c,'value':c} for c in all_counties], multi=True)]),
            html.Div([html.Label("Brand", style=lbl), dcc.Dropdown(id=f"{PERFORMANCE_ID}-brand-filter", options=[{'label':b,'value':b} for b in all_brands], multi=True)]),
            html.Div([html.Label("Category", style=lbl), dcc.Dropdown(id=f"{PERFORMANCE_ID}-category-filter", options=[{'label':c,'value':c} for c in all_categories], multi=True)]),
        ]),
        html.Div(html.Button("Submit", id=f"{PERFORMANCE_ID}-submit-button", n_clicks=0, style=submit_btn), style={'textAlign':'center','marginBottom':'20px'}),
        html.Div(id=f"{PERFORMANCE_ID}-scorecards", style={'display':'flex','justifyContent':'space-around','flexWrap':'wrap','gap':'20px','margin':'40px 0'}),
        dcc.Tabs(children=[
            dcc.Tab(label='📈 COC PERFORMANCE', children=[html.Div([dcc.Graph(id=f"{PERFORMANCE_ID}-daily-achieved-vs-target", style={'width':'48%'}), dcc.Graph(id=f"{PERFORMANCE_ID}-daily-percent-achieved", style={'width':'48%'})], style={'display':'flex','gap':'20px'})]),
            dcc.Tab(label='🏷️ BRAND & COUNTY PERFORMANCE', children=[html.Div([dcc.Graph(id=f"{PERFORMANCE_ID}-client-achievement-bar", style={'width':'48%'}), dcc.Graph(id=f"{PERFORMANCE_ID}-county-achievement-bar", style={'width':'48%'} )], style={'display':'flex','gap':'20px'}), html.Div([dcc.Graph(id=f"{PERFORMANCE_ID}-fr-per-county", style={'width':'48%'}), dcc.Graph(id=f"{PERFORMANCE_ID}-fr-per-title", style={'width':'48%'})], style={'display':'flex','gap':'20px','marginTop':'20px'})]),
            dcc.Tab(label='📈 ASSISTANT COC PERFORMANCE', children=[html.Div([dcc.Graph(id=f"{PERFORMANCE_ID}-assistant-daily-achieved-vs-target", style={'width':'48%'}), dcc.Graph(id=f"{PERFORMANCE_ID}-assistant-daily_PERCENT-achieved", style={'width':'48%'})], style={'display':'flex','gap':'20px'})]),
        ]),
    ], style={'fontFamily':'Segoe UI','padding':'40px','backgroundColor':'#eef2f5'})

def layout_ndume():
    # Generate dummy vaccine data for display
    vax_df = generate_dummy_vaccines()
    fig_vax = px.bar(vax_df.groupby('County')['Number_of_Vaccines'].sum().reset_index(name='Vaccines'), 
                     x='County', y='Vaccines', 
                     title='Vaccines Administered per County (Dummy Data)', 
                     template='plotly_white', text_auto=True)
    fig_vax.update_layout(xaxis_tickangle=-45)

    return html.Div([
        html.Div([html.H1('Ndume Adoptions - Portfolio Demo', style={'textAlign':'center','color':'#004d00','padding':'20px','fontFamily':'Verdana','backgroundColor':'#e6ffe6'}), 
                 html.P("Demonstrating livestock adoption analytics with synthetic data", style={'textAlign':'center','color':'#666','marginBottom':'20px'}),
                 html.Div([dcc.Link(html.Button('Product Adoptions', style=nav_btn), href=f"/{ADOPTION_ID}"), 
                          dcc.Link(html.Button('Performance Tracker', style={**nav_btn,'marginLeft':'10px'}), href=f"/{PERFORMANCE_ID}"), 
                          dcc.Link(html.Button('Comparison Analysis', style={**nav_btn_alt,'marginLeft':'10px'}), href=f"/{COMPARISON_ID}"),
                          dcc.Link(html.Button('Admin/QA Panel', style={**nav_btn, 'backgroundColor':'#8e44ad','marginLeft':'10px'}), href=f"/{SELF_SERVICE_ID}")], style=nav_links)], style={'position':'relative'}),
        html.Div([
            html.Div([
                html.H3("📊 Dummy Data Note", style={'textAlign': 'center', 'color': '#004d00'}),
                html.P("All charts below display randomly generated data for portfolio demonstration purposes.", 
                      style={'textAlign': 'center', 'color': '#666', 'padding': '10px'})
            ], style={'backgroundColor': '#e3f2fd', 'padding': '15px', 'margin': '20px', 'borderRadius': '10px'}),
            
            html.Div(dcc.Graph(figure=fig_nd1), style={'width':'48%','margin':'1%'}), 
            html.Div(dcc.Graph(figure=fig_nd2), style={'width':'48%','margin':'1%'}), 
            html.Div(dcc.Graph(figure=fig_nd3), style={'width':'48%','margin':'1%'}), 
            html.Div(dcc.Graph(figure=fig_vax),  style={'width':'48%','margin':'1%'})
        ], style={'display':'flex','flexWrap':'wrap','justifyContent':'space-between','backgroundColor':'#f0fff0','padding':'20px','borderRadius':'10px'})
    ], style={'backgroundColor':'#e6ffe6','minHeight':'100vh','padding':'20px'})

# Comparison layout
def layout_comparison():
    df1 = get_df1()
    graph_style = {'display':'grid','rowGap':'20px'}
    return html.Div([
        html.Div([html.H1('COMPARISON ANALYSIS - PORTFOLIO DEMO', style={'textAlign':'center','color':'#2c3e50','padding':'20px'}), 
                 html.P("Comparative analytics using synthetic data", style={'textAlign':'center','color':'#666','marginBottom':'20px'}),
                 html.Div([dcc.Link(html.Button('Product Adoptions', style=nav_btn), href=f"/{ADOPTION_ID}"), 
                          dcc.Link(html.Button('Performance Tracker', style={**nav_btn,'marginLeft':'10px'}), href=f"/{PERFORMANCE_ID}"), 
                          dcc.Link(html.Button('Ndume Adoptions', style={**nav_btn_alt,'marginLeft':'10px'}), href=f"/{NDUME_ID}"),
                          dcc.Link(html.Button('Admin/QA Panel', style={**nav_btn, 'backgroundColor':'#8e44ad','marginLeft':'10px'}), href=f"/{SELF_SERVICE_ID}")], style=nav_links)], style={'position':'relative'}),
        html.Div(style=grid_cmp, children=[
            html.Div([html.Label('Current: From', style=lbl), dcc.DatePickerSingle(
                id=f"{COMPARISON_ID}-cur-start", 
                date=get_today(), 
                display_format='DD-MM-YYYY'
            )]),
            html.Div([html.Label('Current: To', style=lbl), dcc.DatePickerSingle(
                id=f"{COMPARISON_ID}-cur-end", 
                date=get_today(), 
                display_format='DD-MM-YYYY'
            )]),
            html.Div([html.Label('Compare: From', style=lbl), dcc.DatePickerSingle(
                id=f"{COMPARISON_ID}-cmp-start", 
                date=(get_today()-datetime.timedelta(days=30)), 
                display_format='DD-MM-YYYY'
            )]),
            html.Div([html.Label('Compare: To', style=lbl), dcc.DatePickerSingle(
                id=f"{COMPARISON_ID}-cmp-end", 
                date=(get_today()-datetime.timedelta(days=1)), 
                display_format='DD-MM-YYYY'
            )]),
            html.Div([html.Label('County', style=lbl), dcc.Dropdown(id=f"{COMPARISON_ID}-county", options=[{'label':c,'value':c} for c in sorted(df1['County'].dropna().unique())], multi=True)]),
            html.Div([html.Label('Brand', style=lbl), dcc.Dropdown(id=f"{COMPARISON_ID}-brand", options=[{'label':c,'value':c} for c in sorted(df1['Brand'].dropna().unique())], multi=True)]),
            html.Div([html.Label('Category', style=lbl), dcc.Dropdown(id=f"{COMPARISON_ID}-cat", options=[{'label':c,'value':c} for c in sorted(df1['Category'].dropna().unique())], multi=True)]),
            html.Div([html.Label('Trainer', style=lbl), dcc.Dropdown(id=f"{COMPARISON_ID}-trainer", options=[{'label':c,'value':c} for c in sorted(df1['Trainer Name'].dropna().unique())], multi=True)]),
            html.Div([html.Label('Status', style=lbl), dcc.Dropdown(id=f"{COMPARISON_ID}-status", options=[{'label':c,'value':c} for c in sorted(df1['Status'].dropna().unique())], multi=True)]),
        ]),
        html.Div(html.Button('Submit', id=f"{COMPARISON_ID}-submit", n_clicks=0, style=submit_btn), style={'textAlign':'center','marginBottom':'20px'}),
        html.Div([html.Div([html.H3('Current Period', style={'textAlign':'center'}), html.Div([dcc.Graph(id=f"{COMPARISON_ID}-brand-cur"), dcc.Graph(id=f"{COMPARISON_ID}-cat-cur"), dcc.Graph(id=f"{COMPARISON_ID}-prod-cur"), dcc.Graph(id=f"{COMPARISON_ID}-county-cur"), dcc.Graph(id=f"{COMPARISON_ID}-trainer-cur"), dcc.Graph(id=f"{COMPARISON_ID}-trend-cur")], style=graph_style)], style={'width':'48%','display':'inline-block','verticalAlign':'top'}), html.Div([html.H3('Comparison Period', style={'textAlign':'center'}), html.Div([dcc.Graph(id=f"{COMPARISON_ID}-brand-cmp"), dcc.Graph(id=f"{COMPARISON_ID}-cat-cmp"), dcc.Graph(id=f"{COMPARISON_ID}-prod-cmp"), dcc.Graph(id=f"{COMPARISON_ID}-county-cmp"), dcc.Graph(id=f"{COMPARISON_ID}-trainer-cmp"), dcc.Graph(id=f"{COMPARISON_ID}-trend-cmp")], style=graph_style)], style={'width':'48%','display':'inline-block','marginLeft':'4%','verticalAlign':'top'})], style={'padding':'20px'})
    ], style={'backgroundColor':'#e6f7ff','minHeight':'100vh','padding':'20px'})

# Router
app.layout = html.Div([dcc.Location(id='url', refresh=False), html.Div(id='page-content')])

@app.callback(Output('page-content','children'), Input('url','pathname'))
def display_page(p):
    if p == f"/{PERFORMANCE_ID}": return layout_performance()
    if p == f"/{NDUME_ID}": return layout_ndume()
    if p == f"/{COMPARISON_ID}": return layout_comparison()
    if p == f"/{SELF_SERVICE_ID}": return layout_login()
    return layout_adoption()

# CELL 3/4 - Complete Callbacks for Role-Based System

# ----------------- Enhanced Authentication Callbacks -----------------

@app.callback(
    [Output('page-content', 'children', allow_duplicate=True),
     Output('login-message', 'children')],
    Input('login-button', 'n_clicks'),
    [State('login-username', 'value'),
     State('login-password', 'value')],
    prevent_initial_call=True
)
def authenticate(n_clicks, username, password):
    if n_clicks:
        if username in USER_CREDENTIALS and USER_CREDENTIALS[username]["password"] == password:
            role = USER_CREDENTIALS[username]["role"]
            if role == "admin":
                return layout_admin(), ""
            else:  # qa role
                return layout_qa(), ""
        else:
            return dash.no_update, html.Div("Invalid credentials", style={'color': 'red', 'fontWeight': 'bold'})
    return dash.no_update, ""

@app.callback(
    Output('page-content', 'children', allow_duplicate=True),
    [Input('logout-button', 'n_clicks')],
    prevent_initial_call=True
)
def logout(n_clicks):
    if n_clicks:
        return layout_login()
    return dash.no_update

# ----------------- QA Progress Dashboard Callback -----------------

@app.callback(
    Output('qa-progress-cards', 'children'),
    [Input('qa-apply-filters', 'n_clicks'),
     Input('qa-refresh', 'n_clicks')],
    [State('qa-start-date', 'date'),
     State('qa-end-date', 'date'),
     State('qa-county-filter', 'value'),
     State('qa-brand-filter', 'value'),
     State('qa-category-filter', 'value'),
     State('qa-trainer-filter', 'value'),
     State('qa-status-filter', 'value')]
)
def update_qa_progress(n_apply, n_refresh, start_date, end_date, counties, brands, categories, trainers, statuses):
    df = get_df1().copy()
    
    # Apply filters
    if start_date:
        df = df[df['Date'] >= pd.to_datetime(start_date)]
    if end_date:
        df = df[df['Date'] <= pd.to_datetime(end_date)]
    if counties:
        df = df[df['County'].isin(counties)]
    if brands:
        df = df[df['Brand'].isin(brands)]
    if categories:
        df = df[df['Category'].isin(categories)]
    if trainers:
        df = df[df['Trainer Name'].isin(trainers)]
    if statuses:
        df = df[df['Status'].isin(statuses)]
    
    # Calculate metrics
    total_records = len(df)
    pending = len(df[df['Status'].isin(['Pending', 'Unverified', 'Undigitized'])])
    verified = len(df[df['Status'] == 'Verified'])
    rejected = len(df[df['Status'] == 'Rejected'])
    
    pending_pct = (pending / total_records * 100) if total_records > 0 else 0
    verified_pct = (verified / total_records * 100) if total_records > 0 else 0
    rejected_pct = (rejected / total_records * 100) if total_records > 0 else 0
    
    cards = [
        html.Div([
            html.H3(f"{total_records:,}", style={'color': '#003366', 'fontSize': '24px', 'margin': '0'}),
            html.P("Total Records", style={'margin': '5px 0', 'color': '#666'})
        ], style={
            'padding': '20px', 'background': 'white', 'borderRadius': '10px', 
            'textAlign': 'center', 'width': '180px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
            'borderLeft': '5px solid #003366'
        }),
        
        html.Div([
            html.H3(f"{pending}", style={'color': '#f39c12', 'fontSize': '24px', 'margin': '0'}),
            html.P(f"Pending ({pending_pct:.1f}%)", style={'margin': '5px 0', 'color': '#666'})
        ], style={
            'padding': '20px', 'background': 'white', 'borderRadius': '10px', 
            'textAlign': 'center', 'width': '180px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
            'borderLeft': '5px solid #f39c12'
        }),
        
        html.Div([
            html.H3(f"{verified}", style={'color': '#27ae60', 'fontSize': '24px', 'margin': '0'}),
            html.P(f"Verified ({verified_pct:.1f}%)", style={'margin': '5px 0', 'color': '#666'})
        ], style={
            'padding': '20px', 'background': 'white', 'borderRadius': '10px', 
            'textAlign': 'center', 'width': '180px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
            'borderLeft': '5px solid #27ae60'
        }),
        
        html.Div([
            html.H3(f"{rejected}", style={'color': '#e74c3c', 'fontSize': '24px', 'margin': '0'}),
            html.P(f"Rejected ({rejected_pct:.1f}%)", style={'margin': '5px 0', 'color': '#666'})
        ], style={
            'padding': '20px', 'background': 'white', 'borderRadius': '10px', 
            'textAlign': 'center', 'width': '180px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)',
            'borderLeft': '5px solid #e74c3c'
        })
    ]
    
    return cards

# ----------------- QA Bulk Actions Callback -----------------

@app.callback(
    [Output('qa-table', 'data', allow_duplicate=True),
     Output('qa-action-message', 'children')],
    [Input('qa-bulk-approve', 'n_clicks'),
     Input('qa-bulk-reject', 'n_clicks')],
    [State('qa-table', 'data'),
     State('qa-table', 'selected_rows')],
    prevent_initial_call=True
)
def handle_qa_bulk_actions(n_approve, n_reject, table_data, selected_rows):
    ctx = dash.callback_context
    if not ctx.triggered:
        return dash.no_update, ""
    
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if not selected_rows:
        return dash.no_update, "⚠️ Please select records to perform bulk action."
    
    df = pd.DataFrame(table_data)
    
    if trigger_id == 'qa-bulk-approve':
        # Bulk approve - clear rejection reasons
        for idx in selected_rows:
            if idx < len(df):
                df.at[idx, 'Status'] = 'Verified'
                df.at[idx, 'Rejection Reason'] = ''  # Clear rejection reason
        message = f"✅ Approved {len(selected_rows)} record(s)"
        
    elif trigger_id == 'qa-bulk-reject':
        # Bulk reject - set default reason
        default_reason = "Bulk rejection - needs review"
        for idx in selected_rows:
            if idx < len(df):
                df.at[idx, 'Status'] = 'Rejected'
                df.at[idx, 'Rejection Reason'] = default_reason
        message = f"❌ Rejected {len(selected_rows)} record(s). Please update rejection reasons in the table."
    
    else:
        return dash.no_update, ""
    
    return df.to_dict('records'), message

# ----------------- QA Save Changes Callback -----------------

@app.callback(
    Output('qa-action-message', 'children', allow_duplicate=True),
    Input('qa-save-changes', 'n_clicks'),
    State('qa-table', 'data'),
    prevent_initial_call=True
)
def save_qa_changes(n_clicks, table_data):
    if n_clicks:
        try:
            df = pd.DataFrame(table_data)
            
            # Ensure proper data types
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
            
            for ncol in ['Amount','Quantity','Actual Sales','Orders']:
                if ncol in df.columns:
                    df[ncol] = pd.to_numeric(df[ncol], errors='coerce').fillna(0)
            
            # Ensure RowID exists
            if 'RowID' not in df.columns:
                df.insert(0, 'RowID', [f"r{int(time.time())}_{i}" for i in range(len(df))])
            
            success, msg = persist_adoptions(df)
            if success:
                # Clear cache to force reload
                if 'df1' in _CACHE:
                    del _CACHE['df1']
                if 'adoptions_perf' in _CACHE:
                    del _CACHE['adoptions_perf']
                    
                return f"✅ Changes saved successfully! {msg}"
            else:
                return f"❌ Failed to save changes: {msg}"
        except Exception as e:
            logger.exception('QA save changes failed')
            return f"❌ Save error: {str(e)}"
    return ""

# ----------------- QA Refresh Data Callback -----------------

@app.callback(
    [Output('qa-table-container', 'children', allow_duplicate=True),
     Output('qa-action-message', 'children', allow_duplicate=True)],
    Input('qa-refresh', 'n_clicks'),
    prevent_initial_call=True
)
def refresh_qa_data(n_clicks):
    if n_clicks:
        table = make_qa_table_component()
        return table, "Data refreshed successfully!"
    return dash.no_update, ""

# ----------------- QA Filter Application Callback -----------------


@app.callback(
    [Output('qa-table-container', 'children'),
     Output('qa-table', 'selected_rows', allow_duplicate=True)],  # Added to clear checkboxes
    Input('qa-apply-filters', 'n_clicks'),
    [State('qa-start-date', 'date'),
     State('qa-end-date', 'date'),
     State('qa-county-filter', 'value'),
     State('qa-brand-filter', 'value'),
     State('qa-category-filter', 'value'),
     State('qa-trainer-filter', 'value'),
     State('qa-status-filter', 'value')],
    prevent_initial_call=True
)
def apply_qa_filters(n_clicks, start_date, end_date, counties, brands, categories, trainers, statuses):
    if n_clicks:
        # Get filtered data
        df = get_df1().copy()
        
        # Apply filters
        if start_date:
            df = df[df['Date'] >= pd.to_datetime(start_date)]
        if end_date:
            df = df[df['Date'] <= pd.to_datetime(end_date)]
        if counties:
            df = df[df['County'].isin(counties)]
        if brands:
            df = df[df['Brand'].isin(brands)]
        if categories:
            df = df[df['Category'].isin(categories)]
        if trainers:
            df = df[df['Trainer Name'].isin(trainers)]
        if statuses:
            df = df[df['Status'].isin(statuses)]
        
        # Create table with filtered data
        if 'RowID' not in df.columns:
            df.insert(0, 'RowID', [f"r{int(time.time())}_{i}" for i in range(len(df))])
        
        # Add Rejection Reason column if not exists
        if 'Rejection Reason' not in df.columns:
            df['Rejection Reason'] = ''
        
        cols = list(df.columns)
        columns = []
        for c in cols:
            col_def = {"name": c, "id": c}
            if c == 'RowID':
                col_def["editable"] = False
                col_def["hideable"] = True
            elif c == 'Status':
                col_def["editable"] = True
                col_def['presentation'] = 'dropdown'
            elif c == 'Rejection Reason':
                col_def["editable"] = True
                col_def["name"] = "Rejection Reason/Comments"
            else:
                col_def["editable"] = False
                
            columns.append(col_def)
        
        table = dash_table.DataTable(
            id='qa-table',
            columns=columns,
            data=df.fillna("").to_dict('records'),
            editable=True,
            row_selectable='multi',
            selected_rows=[],
            page_action='native',
            page_size=15,
            dropdown={
                'Status': {
                    'options': [
                        {"label":"✅ Verified","value":"Verified"},
                        {"label":"⏳ Pending","value":"Pending"},
                        {"label":"❌ Rejected","value":"Rejected"},
                        {"label":"📄 Undigitized","value":"Undigitized"},
                        {"label":"❓ Unverified","value":"Unverified"}
                    ]
                }
            },
            style_table={'overflowX': 'auto', 'borderRadius': '8px'},
            style_cell={
                'minWidth':'120px', 'width':'150px', 'maxWidth':'250px', 
                'whiteSpace':'normal', 'textAlign': 'left',
                'fontFamily': 'Arial, sans-serif'
            },
            style_header={
                'backgroundColor': '#27ae60',
                'color': 'white',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {
                        'filter_query': '{Status} = "Verified"',
                    },
                    'backgroundColor': '#d4edda',
                    'color': '#155724'
                },
                {
                    'if': {
                        'filter_query': '{Status} = "Rejected"',
                    },
                    'backgroundColor': '#f8d7da',
                    'color': '#721c24'
                },
                {
                    'if': {
                        'filter_query': '{Status} = "Pending"',
                    },
                    'backgroundColor': '#fff3cd',
                    'color': '#856404'
                },
                {
                    'if': {'state': 'selected'},
                    'backgroundColor': 'rgba(39, 174, 96, 0.2)',
                    'border': '1px solid #27ae60'
                }
            ],
            tooltip_data=[
                {
                    column: {'value': str(value), 'type': 'markdown'}
                    for column, value in row.items()
                } for row in df.to_dict('records')
            ],
            tooltip_duration=None
        )
        return table, []  # Return empty selected_rows to clear checkboxes
    return dash.no_update, dash.no_update
    
# ----------------- Clear QA Filters Callback -----------------

@app.callback(
    [Output('qa-start-date', 'date'),
     Output('qa-end-date', 'date'),
     Output('qa-county-filter', 'value'),
     Output('qa-brand-filter', 'value'),
     Output('qa-category-filter', 'value'),
     Output('qa-trainer-filter', 'value'),
     Output('qa-status-filter', 'value')],
    Input('qa-clear-filters', 'n_clicks'),
    prevent_initial_call=True
)
def clear_qa_filters(n_clicks):
    if n_clicks:
        return (get_today()-timedelta(days=30),  # start date
                get_today(),                     # end date
                [], [], [], [],                  # clear all multi-selects
                ['Pending', 'Unverified', 'Undigitized'])  # default status
    return dash.no_update

# ----------------- Admin File Management Callbacks -----------------

@app.callback(
    Output('admin-audit-preview', 'children'),
    [Input('admin-upload-data', 'contents'),
     Input('admin-delete-data', 'n_clicks'),
     Input('admin-table', 'data')],  # Add this to refresh when table changes
    prevent_initial_call=True
)
def update_admin_audit_log(upload_contents, delete_clicks, table_data):
    # For demo, show sample audit entries
    sample_entries = [
        f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} | admin | admin_upload_append | r12345 | UPLOAD: upload.csv",
        f"{(datetime.datetime.now() - timedelta(hours=1)).strftime('%Y-%m-%d %H:%M:%S')} | admin | admin_edit | r12345 | Status: Pending → Verified",
        f"{(datetime.datetime.now() - timedelta(hours=2)).strftime('%Y-%m-%d %H:%M:%S')} | qa | qa_bulk_approve | multiple | Status: Pending → Verified",
        f"{(datetime.datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S')} | admin | admin_delete_all | DELETE: all_data",
    ]
    
    items = [html.Div(entry, style={'padding':'4px','borderBottom':'1px solid #eee', 'fontSize': '11px'}) for entry in sample_entries]
    
    return html.Div(items)

@app.callback(
    [Output('admin-table-container', 'children', allow_duplicate=True),
     Output('admin-upload-status', 'children'),
     Output('admin-action-message', 'children')],
    [Input('admin-upload-data', 'contents'),
     Input('admin-delete-data', 'n_clicks')],
    [State('admin-upload-data', 'filename'),
     State('admin-upload-mode', 'value'),
     State('admin-upload-data', 'last_modified')],
    prevent_initial_call=True
)
def handle_admin_file_actions(upload_contents, delete_clicks, upload_filename, upload_mode, upload_date):
    ctx = dash.callback_context
    if not ctx.triggered:
        return dash.no_update, dash.no_update, ""
    
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if trigger_id == 'admin-upload-data' and upload_contents:
        try:
            # Generate new dummy data for upload
            df_up = generate_dummy_adoptions()
            df_up['Upload_Source'] = upload_filename if upload_filename else 'upload.csv'
            
            df_existing = get_df1().copy()
            
            if upload_mode == 'append':
                combined = pd.concat([df_existing, df_up], ignore_index=True)
                operation = "appended to"
            else:
                combined = df_up.copy()
                operation = "replaced with"
            
            success, m = persist_adoptions(combined)
            if success:
                append_audit_entry(
                    action=f'admin_upload_{upload_mode}', 
                    rowid='', 
                    col_changed='UPLOAD', 
                    old='', 
                    new=f'{upload_filename}', 
                    user='admin'
                )
                # Clear cache to force reload
                if 'df1' in _CACHE:
                    del _CACHE['df1']
                if 'adoptions_perf' in _CACHE:
                    del _CACHE['adoptions_perf']
                
                success_status = html.Div([
                    html.I(className="fas fa-check-circle", style={'marginRight': '10px', 'color': '#27ae60', 'fontSize': '24px'}),
                    html.H4("Upload Successful!", style={'color': '#27ae60', 'margin': '10px 0'}),
                    html.P(f"Demo: '{upload_filename}' was {operation} existing data."),
                    html.P(f"Total records: {len(combined):,} (dummy data)"),
                    html.Br(),
                    html.Small(f"Completed at {datetime.datetime.now().strftime('%H:%M:%S')}", style={'color': '#7f8c8d'})
                ], style={
                    'padding': '20px', 
                    'backgroundColor': '#d4edda', 
                    'borderRadius': '8px',
                    'border': '2px solid #27ae60',
                    'textAlign': 'center',
                    'color': '#155724'
                })
                
                table = make_admin_table_component()
                return table, success_status, f"✅ Upload successful! {len(df_up)} dummy records processed"
            else:
                error_status = html.Div([
                    html.I(className="fas fa-exclamation-triangle", style={'marginRight': '10px', 'color': '#e74c3c', 'fontSize': '24px'}),
                    html.H4("Upload Failed", style={'color': '#e74c3c', 'margin': '10px 0'}),
                    html.P(f"Error: {m}"),
                ], style={
                    'padding': '20px', 
                    'backgroundColor': '#f8d7da', 
                    'borderRadius': '8px',
                    'border': '2px solid #e74c3c',
                    'textAlign': 'center',
                    'color': '#721c24'
                })
                return dash.no_update, error_status, f"❌ Upload failed: {m}"
                
        except Exception as e:
            logger.exception('Admin upload failed')
            error_status = html.Div([
                html.I(className="fas fa-exclamation-triangle", style={'marginRight': '10px', 'color': '#e74c3c', 'fontSize': '24px'}),
                html.H4("Upload Error", style={'color': '#e74c3c', 'margin': '10px 0'}),
                html.P(f"Error: {str(e)}"),
            ], style={
                'padding': '20px', 
                'backgroundColor': '#f8d7da', 
                'borderRadius': '8px',
                'border': '2px solid #e74c3c',
                'textAlign': 'center',
                'color': '#721c24'
            })
            return dash.no_update, error_status, f"❌ Upload error: {str(e)}"
    
    elif trigger_id == 'admin-delete-data' and delete_clicks:
        try:
            # Create empty dataframe with same structure
            df_empty = get_df1().copy()
            if not df_empty.empty:
                df_empty = df_empty.iloc[0:0]  # Keep columns, remove all rows
            
            success, m = persist_adoptions(df_empty)
            if success:
                append_audit_entry(
                    action='admin_delete_all', 
                    rowid='', 
                    col_changed='DELETE', 
                    old='', 
                    new='all_data', 
                    user='admin'
                )
                # Clear cache
                if 'df1' in _CACHE:
                    del _CACHE['df1']
                if 'adoptions_perf' in _CACHE:
                    del _CACHE['adoptions_perf']
                    
                delete_status = html.Div([
                    html.I(className="fas fa-check-circle", style={'marginRight': '10px', 'color': '#27ae60', 'fontSize': '24px'}),
                    html.H4("Data Deleted Successfully", style={'color': '#27ae60', 'margin': '10px 0'}),
                    html.P("All adoption data has been cleared (demo)."),
                    html.Br(),
                    html.Small(f"Completed at {datetime.datetime.now().strftime('%H:%M:%S')}", style={'color': '#7f8c8d'})
                ], style={
                    'padding': '20px', 
                    'backgroundColor': '#d4edda', 
                    'borderRadius': '8px',
                    'border': '2px solid #27ae60',
                    'textAlign': 'center',
                    'color': '#155724'
                })
                
                table = make_admin_table_component()
                return table, delete_status, "✅ All data deleted successfully (demo)!"
            else:
                error_status = html.Div([
                    html.I(className="fas fa-exclamation-triangle", style={'marginRight': '10px', 'color': '#e74c3c', 'fontSize': '24px'}),
                    html.H4("Delete Failed", style={'color': '#e74c3c', 'margin': '10px 0'}),
                    html.P(f"Error: {m}"),
                ], style={
                    'padding': '20px', 
                    'backgroundColor': '#f8d7da', 
                    'borderRadius': '8px',
                    'border': '2px solid #e74c3c',
                    'textAlign': 'center',
                    'color': '#721c24'
                })
                return dash.no_update, error_status, f"❌ Delete failed: {m}"
        except Exception as e:
            error_status = html.Div([
                html.I(className="fas fa-exclamation-triangle", style={'marginRight': '10px', 'color': '#e74c3c', 'fontSize': '24px'}),
                html.H4("Delete Error", style={'color': '#e74c3c', 'margin': '10px 0'}),
                html.P(f"Error: {str(e)}"),
            ], style={
                'padding': '20px', 
                'backgroundColor': '#f8d7da', 
                'borderRadius': '8px',
                'border': '2px solid #e74c3c',
                'textAlign': 'center',
                'color': '#721c24'
            })
            return dash.no_update, error_status, f"❌ Delete error: {str(e)}"
    
    return dash.no_update, dash.no_update, ""

# ----------------- Admin Table Save Callback -----------------

@app.callback(
    Output('admin-action-message', 'children', allow_duplicate=True),
    Input('admin-table', 'data'),
    State('admin-table', 'data_previous'),
    prevent_initial_call=True
)
def handle_admin_table_edits(current_data, previous_data):
    if previous_data is None:
        return ""
    
    # Find what changed
    current_df = pd.DataFrame(current_data)
    previous_df = pd.DataFrame(previous_data)
    
    # Persist changes
    success, msg = persist_adoptions(current_df)
    if success:
        # Log changes to audit
        for idx, (current_row, previous_row) in enumerate(zip(current_data, previous_data)):
            for key in current_row:
                if current_row[key] != previous_row[key]:
                    append_audit_entry(
                        action='admin_edit', 
                        rowid=current_row.get('RowID', f'row_{idx}'),
                        col_changed=key,
                        old=previous_row[key],
                        new=current_row[key],
                        user='admin'
                    )
        return "Changes auto-saved successfully!"
    else:
        return f"Save failed: {msg}"

# ----------------- Original Adoption Dashboard Callback -----------------

@app.callback(
    [
        Output(f"{ADOPTION_ID}-total-amount","children"),
        Output(f"{ADOPTION_ID}-verified-percentage","children"),
        Output(f"{ADOPTION_ID}-pending-percentage","children"),
        Output(f"{ADOPTION_ID}-rejected-percentage","children"),
        Output(f"{ADOPTION_ID}-total-target","children"),
        Output(f"{ADOPTION_ID}-change-percentage","children"),
        Output(f"{ADOPTION_ID}-timeline-note","children"),
        Output(f"{ADOPTION_ID}-amount-per-brand","figure"),
        Output(f"{ADOPTION_ID}-amount-per-category","figure"),
        Output(f"{ADOPTION_ID}-amount-per-product","figure"),
        Output(f"{ADOPTION_ID}-amount-per-county","figure"),
        Output(f"{ADOPTION_ID}-amount-per-trainer","figure"),
        Output(f"{ADOPTION_ID}-amount-per-day","figure"),
        Output(f"{ADOPTION_ID}-sistema-total-qty","children"),
        Output(f"{ADOPTION_ID}-sistema-card","style"),
        Output(f"{ADOPTION_ID}-sistema-charts-container","style"),
        Output(f"{ADOPTION_ID}-sistema-qty-per-product","figure"),
        Output(f"{ADOPTION_ID}-sistema-installed-per-county","figure"),
    ],
    Input(f"{ADOPTION_ID}-submit-button","n_clicks"),
    [
        State(f"{ADOPTION_ID}-start-date","date"),
        State(f"{ADOPTION_ID}-end-date","date"),
        State(f"{ADOPTION_ID}-county-filter","value"),
        State(f"{ADOPTION_ID}-brand-filter","value"),
        State(f"{ADOPTION_ID}-category-filter","value"),
        State(f"{ADOPTION_ID}-trainer-filter","value"),
        State(f"{ADOPTION_ID}-status-filter","value"),
    ],
    prevent_initial_call=False
)
def update_adoption(n_clicks, start_date, end_date, counties, brands, categories, trainers, statuses):
    targets_df = get_targets_df()
    df1 = get_df1()
    df_loans = get_df_loans()

    # safe date parsing
    start_dt = pd.to_datetime(start_date) if start_date else pd.to_datetime(get_today()-timedelta(days=30))
    end_dt   = pd.to_datetime(end_date)   if end_date   else pd.to_datetime(get_today())
    months = pd.date_range(start_dt.replace(day=1), end_dt.replace(day=1), freq='MS').strftime('%B').tolist()

    years = [str(y) for y in range(start_dt.year, end_dt.year + 1)]

    tf = targets_df[targets_df['Month'].isin(months) & targets_df['Year'].isin(years)].copy()
    if counties:
        tf = tf[tf['County'].isin(counties)]
    if brands:
        tf = tf[tf['Client'].isin(brands)]
    if trainers:
        tf = tf[tf['Co-ordinator'].isin(trainers)]
    if categories and 'Category' in tf.columns:
        sel_cat_l = [str(x).strip().lower() for x in categories]
        tf = tf[tf['Category'].astype(str).str.strip().str.lower().isin(sel_cat_l)]

    total_target = tf['Cumulative Target Per Month'].sum() / 2 if 'Cumulative Target Per Month' in tf.columns else 0

    dff = df1.copy()
    if start_date: dff = dff[dff['Date'] >= start_dt]
    if end_date:   dff = dff[dff['Date'] <= end_dt]
    if counties:   dff = dff[dff['County'].isin(counties)]
    if brands:     dff = dff[dff['Brand'].isin(brands)]
    if categories: dff = dff[dff['Category'].isin(categories)]
    if trainers:   dff = dff[dff['Trainer Name'].isin(trainers)]
    if statuses:   dff = dff[dff['Status'].isin(statuses)]

    sum_amt = dff['Amount'].sum()
    verified_amt = dff[dff['Status'].str.strip().str.lower() == 'verified']['Amount'].sum() if 'Status' in dff.columns else 0
    verified_pct = (verified_amt / sum_amt * 100) if sum_amt > 0 else 0
    pct_of_target = (sum_amt / total_target * 100) if total_target > 0 else 0

    total_amt_display = f"KES {sum_amt:,.2f} ({pct_of_target:.1f}% of target)"
    verified_str = f"{verified_pct:.2f}% Verified"

    pending_statuses = {'undigitized','unverified','pending'}
    rejected_statuses = {'rejected','declined'}

    if 'Status' in dff.columns:
        status_s = dff['Status'].astype(str).str.strip().str.lower()
        pending_amt = dff.loc[status_s.isin(pending_statuses), 'Amount'].sum()
        rejected_amt = dff.loc[status_s.isin(rejected_statuses), 'Amount'].sum()
    else:
        pending_amt = 0
        rejected_amt = 0

    pending_pct = (pending_amt / sum_amt * 100) if sum_amt > 0 else 0
    rejected_pct = (rejected_amt / sum_amt * 100) if sum_amt > 0 else 0

    pending_str = f"{pending_pct:.2f}% Pending"
    rejected_str = f"{rejected_pct:.2f}% Rejected"

    change_pct = 'No Data'
    timeline_note = ''
    if not dff.empty and 'Date' in dff.columns and dff['Date'].min() != dff['Date'].max():
        daily = dff.groupby('Date')['Amount'].sum().sort_index()
        if len(daily) > 1:
            init, fin = daily.iloc[0], daily.iloc[-1]
            if init != 0:
                pct = (fin - init) / init * 100
                direction = 'Increase' if pct > 0 else 'Decrease' if pct < 0 else 'No Change'
                change_pct = f"{abs(pct):.2f}% {direction}"
        days = (end_dt - start_dt).days
        timeline_note = (
            'Daily percentage change' if days <= 7 else
            'Weekly percentage change' if days < 30 else
            'Monthly percentage change' if days <= 90 else
            'Quarterly percentage change'
        )

    bd = dff.groupby('Brand').agg({'Actual Sales':'sum','Orders':'sum'}).reset_index() if 'Actual Sales' in dff.columns and 'Orders' in dff.columns else pd.DataFrame({'Brand':[]})
    if not bd.empty:
        bd['Total'] = bd['Actual Sales'] + bd['Orders']
        bd_m = bd.sort_values('Total', ascending=False).melt(id_vars='Brand', value_vars=['Actual Sales','Orders'], var_name='Metric', value_name='Value')
        fig_brand = px.funnel(bd_m, x='Value', y='Brand', color='Metric', title='Actual Sales & Orders Per Client (Dummy Data)', template='plotly_white')
    else:
        fig_brand = px.scatter(title='Actual Sales & Orders Per Client')

    fig_cat = px.pie(dff, values='Amount', names='Category', hole=0.4, title='Amount per Category (Dummy Data)', template='plotly_white') if not dff.empty else px.scatter(title='Amount per Category')
    prod_df = dff.groupby('Product')['Amount'].sum().reset_index().sort_values('Amount') if 'Product' in dff.columns else pd.DataFrame({'Product':[]})
    fig_prod = px.bar(prod_df, x='Amount', y='Product', orientation='h', title='Amount per Product (Dummy Data)', template='plotly_white') if not prod_df.empty else px.scatter(title='Amount per Product')
    county_df = dff.groupby('County')['Amount'].sum().reset_index().sort_values('Amount') if 'County' in dff.columns else pd.DataFrame({'County':[]})
    fig_cnt = px.bar(county_df, x='Amount', y='County', orientation='h', title='Amount per County (Dummy Data)', template='plotly_white') if not county_df.empty else px.scatter(title='Amount per County')
    tr_df = dff.groupby('Trainer Name')['Amount'].sum().reset_index().merge(df_loans, on='Trainer Name', how='left').fillna(0) if 'Trainer Name' in dff.columns else pd.DataFrame({'Trainer Name':[]})
    fig_tr = px.bar(tr_df, x='Trainer Name', y=['Amount','Loan Borrowed','Loan Repaid'], barmode='stack', title='Adoption and Loans Per Trainer (Dummy Data)', template='plotly_white') if not tr_df.empty else px.scatter(title='Adoption and Loans Per Trainer')
    if hasattr(fig_tr,'update_layout'):
        fig_tr.update_layout(xaxis_tickangle=-90)
    day_df = dff.groupby('Date')['Amount'].sum().reset_index().sort_values('Date') if 'Date' in dff.columns else pd.DataFrame({'Date':[]})
    fig_day = px.line(day_df, x='Date', y='Amount', markers=True, title='Daily Adoption Trend (Dummy Data)', template='plotly_white') if not day_df.empty else px.scatter(title='Daily Adoption Trend')

    # SISTEMA logic
    default_date_str = get_today().strftime('%Y-%m-%d')
    date_filter_used = False
    try:
        if start_date and end_date:
            date_filter_used = (str(start_date) != default_date_str) or (str(end_date) != default_date_str)
    except Exception:
        date_filter_used = False

    hide_due_to_other_clients = False
    if brands:
        if any('sistema' not in (str(b).lower()) for b in brands):
            hide_due_to_other_clients = True

    show_sistema_charts = bool((brands and len(brands) > 0) or (counties and len(counties) > 0) or (trainers and len(trainers) > 0) or date_filter_used)

    sistema_card_visible_style = {'textAlign':'center','padding':'20px','background':'#fff7ff','borderRadius':'12px','width':'320px','boxShadow':'2px 2px 8px rgba(0,0,0,0.08)','margin':'20px','display':'flex','flexDirection':'column','alignItems':'center'}
    sistema_card_hidden_style = {'display':'none'}
    sistema_charts_hidden_style = {'display':'none'}
    sistema_charts_visible_style = {'display':'flex','justifyContent':'center','marginTop':'10px'}

    if hide_due_to_other_clients:
        sistema_total_disp = ''
        sistema_card_style = sistema_card_hidden_style
        sistema_charts_style = sistema_charts_hidden_style
        empty_fig_prod = px.scatter(title='Sistema Bio — Installed Units')
        empty_fig_inst = px.scatter(title='Sistema Bio — Installed Per County')
        fig_s_prod = empty_fig_prod
        fig_s_inst = empty_fig_inst
    else:
        sb = get_df1().copy()
        if start_date and 'Date' in sb.columns:
            sb = sb[sb['Date'] >= start_dt]
        if end_date and 'Date' in sb.columns:
            sb = sb[sb['Date'] <= end_dt]
        sb['Brand_norm'] = sb.get('Brand', pd.Series('', index=sb.index)).astype(str).str.strip().str.lower()
        sb = sb[sb['Brand_norm'].str.contains('sistema', na=False)].copy()
        qty_col = None
        for c in sb.columns:
            if c and c.strip().lower() in ('quantity','qty','unit'):
                qty_col = c
                break
        if qty_col and qty_col != 'Quantity':
            sb = sb.rename(columns={qty_col: 'Quantity'})
        if 'Quantity' in sb.columns:
            sb['Quantity'] = pd.to_numeric(sb['Quantity'], errors='coerce').fillna(0)
        else:
            sb['Quantity'] = 0
        if counties:
            sb = sb[sb['County'].isin(counties)]
        if trainers:
            sb = sb[sb['Trainer Name'].isin(trainers)]
        if statuses:
            sb = sb[sb['Status'].isin(statuses)]
        total_qty = int(sb['Quantity'].sum())
        sistema_total_disp = f"{total_qty:,} unit(s)"
        prod_df_s = sb.groupby('Product')['Quantity'].sum().reset_index().sort_values('Quantity', ascending=False) if 'Product' in sb.columns else pd.DataFrame({'Product':[]})
        fig_s_prod = px.bar(prod_df_s, x='Product', y='Quantity', title='Sistema Bio — Installed Units (Dummy Data)', template='plotly_white') if not prod_df_s.empty else px.scatter(title='Sistema Bio — Installed Units')
        if hasattr(fig_s_prod,'update_layout'):
            fig_s_prod.update_layout(xaxis_tickangle=-45)
        inst_df = sb.groupby('County')['Quantity'].sum().reset_index().sort_values('Quantity', ascending=False) if 'County' in sb.columns else pd.DataFrame({'County':[]})
        fig_s_inst = px.bar(inst_df, x='County', y='Quantity', title='Sistema Bio — Installed Per County (Dummy Data)', template='plotly_white') if not inst_df.empty else px.scatter(title='Sistema Bio — Installed Per County')
        if hasattr(fig_s_inst,'update_layout'):
            fig_s_inst.update_layout(xaxis_tickangle=-45)
        sistema_card_style = sistema_card_visible_style
        sistema_charts_style = sistema_charts_visible_style if show_sistema_charts else sistema_charts_hidden_style

    return (
        total_amt_display,
        verified_str,
        pending_str,
        rejected_str,
        f"KES {total_target:,.2f}",
        change_pct,
        timeline_note,
        fig_brand,
        fig_cat,
        fig_prod,
        fig_cnt,
        fig_tr,
        fig_day,
        sistema_total_disp,
        sistema_card_style,
        sistema_charts_style,
        fig_s_prod,
        fig_s_inst
    )

# ----------------- CSV Download callback for adoption -----------------
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("download-button", "n_clicks"),
    State(f"{ADOPTION_ID}-start-date", "date"),
    State(f"{ADOPTION_ID}-end-date", "date"),
    State(f"{ADOPTION_ID}-county-filter", "value"),
    State(f"{ADOPTION_ID}-brand-filter", "value"),
    State(f"{ADOPTION_ID}-category-filter", "value"),
    State(f"{ADOPTION_ID}-trainer-filter", "value"),
    State(f"{ADOPTION_ID}-status-filter", "value"),
    prevent_initial_call=True
)
def download_adoption_data(n_clicks, start_date, end_date, counties, brands, categories, trainers, statuses):
    dff = get_df1().copy()
    if start_date: dff = dff[dff['Date'] >= pd.to_datetime(start_date)]
    if end_date:   dff = dff[dff['Date'] <= pd.to_datetime(end_date)]
    if counties:   dff = dff[dff['County'].isin(counties)]
    if brands:     dff = dff[dff['Brand'].isin(brands)]
    if categories: dff = dff[dff['Category'].isin(categories)]
    if trainers:   dff = dff[dff['Trainer Name'].isin(trainers)]
    if statuses:   dff = dff[dff['Status'].isin(statuses)]
    
    # Add demo note to downloaded data
    if 'Note' not in dff.columns:
        dff['Note'] = 'Portfolio Demo - Dummy Data'
    
    return dcc.send_data_frame(dff.to_csv, filename='Adoption_Demo_Data.csv', index=False)

# ----------------- PERFORMANCE CALLBACK -----------------
@app.callback(
    [
        Output(f"{PERFORMANCE_ID}-scorecards",               "children"),
        Output(f"{PERFORMANCE_ID}-daily-achieved-vs-target","figure"),
        Output(f"{PERFORMANCE_ID}-daily-percent-achieved",  "figure"),
        Output(f"{PERFORMANCE_ID}-client-achievement-bar",  "figure"),
        Output(f"{PERFORMANCE_ID}-county-achievement-bar",  "figure"),
        Output(f"{PERFORMANCE_ID}-fr-per-county",           "figure"),
        Output(f"{PERFORMANCE_ID}-fr-per-title",            "figure"),
        Output(f"{PERFORMANCE_ID}-assistant-daily-achieved-vs-target","figure"),
        Output(f"{PERFORMANCE_ID}-assistant-daily_PERCENT-achieved",  "figure"),
    ],
    Input(f"{PERFORMANCE_ID}-submit-button", "n_clicks"),
    [
        State(f"{PERFORMANCE_ID}-start-date",        "date"),
        State(f"{PERFORMANCE_ID}-end-date",          "date"),
        State(f"{PERFORMANCE_ID}-coordinator-filter","value"),
        State(f"{PERFORMANCE_ID}-county-filter",     "value"),
        State(f"{PERFORMANCE_ID}-brand-filter",      "value"),
        State(f"{PERFORMANCE_ID}-category-filter",   "value"),
    ]
)
def update_performance(n_clicks, start_date, end_date, sel_coords, sel_counties, sel_brands, sel_categories):
    import pandas as pd
    from dash import html
    import plotly.express as px
    targets_df = get_targets_df()
    adoptions_df = get_adoptions_df()
    assistant_df = get_assistant_df()
    start      = pd.to_datetime(start_date)
    end        = pd.to_datetime(end_date)
    days_count = (end - start).days + 1
    months = pd.date_range(start.replace(day=1), end.replace(day=1), freq='MS').strftime('%B').tolist()
    years = [str(y) for y in range(start.year, end.year + 1)]
    assistant_coords = assistant_df['Co-ordinator'].unique().tolist() if not assistant_df.empty else []
    coords = sel_coords or []
    main_coords = [c for c in coords if c not in assistant_coords]
    assistant_sel = [c for c in coords if c in assistant_coords]
    is_assistant_only = bool(assistant_sel and not main_coords)
    tgt_all  = targets_df[targets_df['Month'].isin(months) & targets_df['Year'].isin(years)].copy()
    tgt_main = tgt_all[~tgt_all['Co-ordinator'].isin(assistant_coords)].copy()
    tgt_asst = tgt_all[tgt_all['Co-ordinator'].isin(assistant_coords)].copy()

    def apply_filters_to_targets(df):
        if sel_counties:
            df = df[df['County'].astype(str).str.strip().isin(sel_counties)]
        if sel_brands:
            df = df[df['Client'].astype(str).str.strip().isin(sel_brands)]
        if sel_categories and 'Category' in df.columns:
            sel_cat_l = [str(x).strip().lower() for x in sel_categories]
            df = df[df['Category'].astype(str).str.strip().str.lower().isin(sel_cat_l)]
        return df

    tgt_main = apply_filters_to_targets(tgt_main)
    tgt_asst = apply_filters_to_targets(tgt_asst)
    if main_coords:
        tgt_main = tgt_main[tgt_main['Co-ordinator'].isin(main_coords)]
    if assistant_sel:
        tgt_asst = tgt_asst[tgt_asst['Co-ordinator'].isin(assistant_sel)]

    ads = adoptions_df[(adoptions_df.Date >= start) & (adoptions_df.Date <= end)].copy()
    if sel_counties:   ads = ads[ads['County'].isin(sel_counties)]
    if sel_brands:     ads = ads[ads['Brand'].isin(sel_brands)]
    if sel_categories: ads = ads[ads['Category'].isin(sel_categories)]
    ads = ads.merge(targets_df[['County','Client','Co-ordinator','Category']].drop_duplicates(), left_on=['County','Brand','Category'], right_on=['County','Client','Category'], how='left')
    ads = ads[~ads['Co-ordinator'].isin(assistant_coords)]
    if main_coords:
        ads = ads[ads['Co-ordinator'].isin(main_coords)]

    asst = assistant_df[(assistant_df.Date >= start) & (assistant_df.Date <= end)].copy()
    if assistant_sel:
        asst = asst[asst['Co-ordinator'].isin(assistant_sel)]
    if sel_counties:
        asst = asst[asst['County'].isin(sel_counties)]

    at  = fetch_active_trainers(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'))
    vax = fetch_vaccines(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'))
    fr  = fetch_farmer_reach(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'))
    if sel_counties:
        at  = at[ at['County'].isin(sel_counties)]
        vax = vax[vax['County'].isin(sel_counties)]
        fr  = fr[ fr['County'].isin(sel_counties)]

    cm      = tgt_main[['County','Co-ordinator']].drop_duplicates()
    at_main = at.merge(cm, on='County', how='left')
    if main_coords:
        at_main = at_main[at_main['Co-ordinator'].isin(main_coords)]
    fr_main = fr.merge(cm, on='County', how='left')
    if main_coords:
        fr_main = fr_main[fr_main['Co-ordinator'].isin(main_coords)]

    if is_assistant_only:
        total_tgt = tgt_asst['Cumulative Target Per Month'].sum() if 'Cumulative Target Per Month' in tgt_asst.columns else 0
        total_ach = int(asst['Achieved'].sum()) if 'Achieved' in asst.columns else 0
        fr_asst   = fr.merge(tgt_asst[['County','Co-ordinator']].drop_duplicates(), on='County', how='inner')
        total_fr  = int(fr_asst['farmer_reach'].sum()) if 'farmer_reach' in fr_asst.columns else 0
    else:
        total_tgt = tgt_main['Cumulative Target Per Month'].sum() if 'Cumulative Target Per Month' in tgt_main.columns else 0
        total_ach = int(ads['Amount'].sum()) if 'Amount' in ads.columns else 0
        total_fr  = int(fr_main['farmer_reach'].sum()) if 'farmer_reach' in fr_main.columns else 0

    total_act = at_main['TrainerID'].nunique() if 'TrainerID' in at_main.columns else 0
    total_vax = int(vax['Number_of_Vaccines'].sum()) if 'Number_of_Vaccines' in vax.columns else 0
    pct_ach   = (total_ach / total_tgt * 100) if total_tgt > 0 else 0

    cards = []
    for val, label, show_pct in [
        (total_tgt, 'Total Target', False),
        (total_ach,'Total Achieved',   True),
        (total_act,'Active Trainers',  False),
        (total_fr, 'Farmer Reach',     False),
        (total_vax,'Vaccines Administered',False),
    ]:
        disp = f"{int(val):,} ({pct_ach:.1f}%)" if show_pct else f"{int(val):,}"
        cards.append(html.Div([html.H3(disp), html.P(label)], style={'padding':'15px','borderRadius':'8px','boxShadow':'0 2px 4px rgba(0,0,0,0.1)','textAlign':'center','width':'180px','backgroundColor':'#c081eb'}))

    df_sum    = ads.groupby('Co-ordinator')['Amount'].sum().reset_index(name='Achieved')
    daily_tgt = tgt_main.groupby('Co-ordinator')['Daily Target'].sum().reset_index(name='SumDaily')
    df_t      = daily_tgt.assign(Target=lambda d: d['SumDaily'] * days_count)[['Co-ordinator','Target']]
    dfm       = df_sum.merge(df_t, on='Co-ordinator', how='left').fillna(0)
    dfm['Pct'] = (dfm['Achieved'] / dfm['Target'] * 100).fillna(0).round(1)

    fig1 = px.bar(dfm.melt(id_vars=['Co-ordinator','Pct'], value_vars=['Target','Achieved']), x='value', y='Co-ordinator', color='variable', barmode='group', title='Targets vs. Achieved by Co-ordinator (Dummy Data)', template='plotly_white')
    for trace in fig1.data:
        if trace.name == 'Achieved':
            pct_map = dfm.set_index('Co-ordinator')['Pct'].to_dict()
            trace.customdata = [[pct_map[y]] for y in trace.y]
            trace.hovertemplate = '%{y}<br>Achieved: %{x:,.0f}<br>%{customdata[0]:.1f}%<extra></extra>'
        else:
            trace.hovertemplate = '%{y}<br>Target: %{x:,.0f}<extra></extra>'

    dates_df = pd.DataFrame({'Date': pd.date_range(start, end)})
    daily    = ads.groupby(['Date','Co-ordinator'])['Amount'].sum().reset_index()
    daily_t  = (dates_df.assign(key=1).merge(df_t.assign(key=1).assign(Daily=lambda d: d['Target']/len(dates_df)), on='key').drop('key', axis=1))
    ds = daily_t.merge(daily, on=['Date','Co-ordinator'], how='left').fillna({'Amount':0})
    ds['PctAch'] = (ds['Amount']/ds['Daily']*100).fillna(0)

    fig2 = px.line(ds, x='Date', y='PctAch', color='Co-ordinator', title='Daily % of Target Achieved (Dummy Data)', template='plotly_white')

    client_ct  = tgt_main.groupby('Client')['Cumulative Target Per Month'].sum().reset_index(name='Target')
    client_ach = ads.groupby('Brand')['Amount'].sum().reset_index(name='Achieved')
    df3        = client_ct.merge(client_ach, left_on='Client', right_on='Brand', how='left').fillna(0)

    fig3 = px.bar(df3, y='Client', x=['Target','Achieved'], orientation='h', barmode='group', title='Brand Achievement (Dummy Data)', template='plotly_white')
    fig3.update_layout(yaxis={'categoryorder':'total ascending'})

    county_ct  = tgt_main.groupby('County')['Cumulative Target Per Month'].sum().reset_index(name='Target')
    county_ach = ads.groupby('County')['Amount'].sum().reset_index(name='Achieved')
    cs2        = county_ct.merge(county_ach, on='County', how='left').fillna(0)

    fig4 = px.bar(cs2, x='County', y=['Target','Achieved'], barmode='group', title='County Achievement (Dummy Data)', template='plotly_white')
    fig4.update_layout(xaxis_tickangle=-45)

    df5 = fr_main.groupby('County')['farmer_reach'].sum().reset_index()
    fig5 = px.bar(df5, x='farmer_reach', y='County', orientation='h', title='Farmer Reach per County (Dummy Data)', template='plotly_white')

    df6 = fr.groupby('Topic_name')['farmer_reach'].sum().reset_index().sort_values('farmer_reach')
    fig6 = px.bar(df6, x='farmer_reach', y='Topic_name', orientation='h', title='Farmer Reach by Topic (Dummy Data)', template='plotly_white')
    fig6.update_layout(yaxis={'categoryorder':'total ascending'})

    df_sum_asst    = asst.groupby('Co-ordinator')['Amount'].sum().reset_index() if 'Amount' in asst.columns else pd.DataFrame({'Co-ordinator':[]})
    daily_tgt_asst = tgt_asst.groupby('Co-ordinator')['Daily Target'].sum().reset_index(name='SumDaily') if 'Daily Target' in tgt_asst.columns else pd.DataFrame({'Co-ordinator':[]})
    df_t_asst      = daily_tgt_asst.assign(Target=lambda d: d['SumDaily'] * days_count)[['Co-ordinator','Target']] if not daily_tgt_asst.empty else pd.DataFrame({'Co-ordinator':[]})
    dfm_asst       = df_sum_asst.merge(df_t_asst, on='Co-ordinator', how='left').fillna(0)
    if 'Amount' in dfm_asst.columns:
        dfm_asst['Pct']= (dfm_asst['Amount'] / dfm_asst['Target'] * 100).fillna(0).round(1)
    else:
        dfm_asst['Pct'] = 0

    fig7 = px.bar(dfm_asst.melt(id_vars=['Co-ordinator','Pct'], value_vars=[c for c in dfm_asst.columns if c in ['Target','Amount']]), x='Co-ordinator', y='value', color='variable', barmode='group', title='Assistant Co-ordinator Monthly (Dummy Data)', template='plotly_white') if not dfm_asst.empty else px.scatter(title='Assistant Co-ordinator Monthly')

    for trace in getattr(fig7, 'data', []):
        if trace.name == 'Amount':
            pct_map = dfm_asst.set_index('Co-ordinator')['Pct'].to_dict()
            trace.customdata = [[pct_map.get(x,0)] for x in trace.x]
            trace.hovertemplate = '%{x}<br>Achieved: %{y:,.0f}<br>%{customdata[0]:.1f}%<extra></extra>'
        else:
            trace.hovertemplate = '%{x}<br>Target: %{y:,.0f}<extra></extra>'

    days_asst = pd.DataFrame({'Date': pd.date_range(start, end)})
    asst_daily= days_asst.merge(asst.groupby('Date')['Amount'].sum().reset_index(), on='Date', how='left').fillna(0)
    tot_tgt_asst = dfm_asst['Target'].sum() if 'Target' in dfm_asst.columns else 0
    asst_daily['PctAch'] = (asst_daily['Amount']/tot_tgt_asst*100).fillna(0) if tot_tgt_asst > 0 else 0

    fig8 = px.line(asst_daily, x='Date', y='PctAch', title='Assistant % of Target Over Time (Dummy Data)', template='plotly_white')

    return cards, fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8

# ----------------- COMPARISON CALLBACK -----------------
@app.callback(
    [
      Output(f"{COMPARISON_ID}-brand-cur","figure"),
      Output(f"{COMPARISON_ID}-cat-cur","figure"),
      Output(f"{COMPARISON_ID}-prod-cur","figure"),
      Output(f"{COMPARISON_ID}-county-cur","figure"),
      Output(f"{COMPARISON_ID}-trainer-cur","figure"),
      Output(f"{COMPARISON_ID}-trend-cur","figure"),
      Output(f"{COMPARISON_ID}-brand-cmp","figure"),
      Output(f"{COMPARISON_ID}-cat-cmp","figure"),
      Output(f"{COMPARISON_ID}-prod-cmp","figure"),
      Output(f"{COMPARISON_ID}-county-cmp","figure"),
      Output(f"{COMPARISON_ID}-trainer-cmp","figure"),
      Output(f"{COMPARISON_ID}-trend-cmp","figure"),
    ],
    Input(f"{COMPARISON_ID}-submit","n_clicks"),
    [
      State(f"{COMPARISON_ID}-cur-start","date"),
      State(f"{COMPARISON_ID}-cur-end","date"),
      State(f"{COMPARISON_ID}-cmp-start","date"),
      State(f"{COMPARISON_ID}-cmp-end","date"),
      State(f"{COMPARISON_ID}-county","value"),
      State(f"{COMPARISON_ID}-brand","value"),
      State(f"{COMPARISON_ID}-cat","value"),
      State(f"{COMPARISON_ID}-trainer","value"),
      State(f"{COMPARISON_ID}-status","value"),
    ]
)
def update_comparison(n, cs, ce, ms, me, counties, brands, cats, trainers, statuses):
    df1 = get_df1()
    def filt(df,s,e):
        d=df[(df['Date']>=pd.to_datetime(s))&(df['Date']<=pd.to_datetime(e))]
        if counties: d=d[d['County'].isin(counties)]
        if brands:   d=d[d['Brand'].isin(brands)]
        if cats:     d=d[d['Category'].isin(cats)]
        if trainers:d=d[d['Trainer Name'].isin(trainers)]
        if statuses:d=d[d['Status'].isin(statuses)]
        return d
    cur=filt(df1,cs,ce)
    cmp=filt(df1,ms,me)
    def make_figs(d):
        bd=d.groupby('Brand').agg({'Actual Sales':'sum','Orders':'sum'}).reset_index() if 'Actual Sales' in d.columns and 'Orders' in d.columns else pd.DataFrame()
        if not bd.empty:
            bd['Total']=bd['Actual Sales']+bd['Orders']
            bd_m=bd.sort_values('Total',ascending=False).melt(id_vars='Brand',value_vars=['Actual Sales','Orders'],var_name='Metric',value_name='Value')
            fig_b=px.funnel(bd_m,x='Value',y='Brand',color='Metric',title='Actual Sales & Orders Per Client (Dummy Data)',template='plotly_white')
        else:
            fig_b=px.scatter(title='Actual Sales & Orders Per Client')
        fig_c=px.pie(d,values='Amount',names='Category',hole=0.4,title='Amount Per Category (Dummy Data)',template='plotly_white') if not d.empty else px.scatter(title='Amount Per Category')
        prod=d.groupby('Product')['Amount'].sum().reset_index().sort_values('Amount') if 'Product' in d.columns else pd.DataFrame()
        fig_p=px.bar(prod,x='Amount',y='Product',orientation='h',title='Amount Per Product (Dummy Data)',template='plotly_white') if not prod.empty else px.scatter(title='Amount Per Product')
        cnt=d.groupby('County')['Amount'].sum().reset_index().sort_values('Amount') if 'County' in d.columns else pd.DataFrame()
        fig_cnt=px.bar(cnt,x='Amount',y='County',orientation='h',title='Amount Per County (Dummy Data)',template='plotly_white') if not cnt.empty else px.scatter(title='Amount Per County')
        tr=d.groupby('Trainer Name')['Amount'].sum().reset_index().merge(get_df_loans(),on='Trainer Name',how='left').fillna(0) if 'Trainer Name' in d.columns else pd.DataFrame()
        fig_t=px.bar(tr,x='Trainer Name',y=['Amount','Loan Borrowed','Loan Repaid'],barmode='stack',title='Adoption and Loans Per Trainer (Dummy Data)',template='plotly_white') if not tr.empty else px.scatter(title='Adoption and Loans Per Trainer')
        if hasattr(fig_t, 'update_layout'):
            fig_t.update_layout(xaxis_tickangle=-90)
        daily=d.groupby('Date')['Amount'].sum().reset_index().sort_values('Date') if 'Date' in d.columns else pd.DataFrame()
        fig_tr=px.line(daily,x='Date',y='Amount',markers=True,title='Daily Adoption Trend (Dummy Data)',template='plotly_white') if not daily.empty else px.scatter(title='Daily Adoption Trend')
        return fig_b,fig_c,fig_p,fig_cnt,fig_t,fig_tr
    return (*make_figs(cur),*make_figs(cmp))

# CELL 4/4
# Run block and any remaining utilities

# A lightweight search/filter helper used in callbacks
def safe_unique_sorted(series):
    try:
        return sorted(series.dropna().unique())
    except Exception:
        return []

# Run block
if __name__ == '__main__':
    logger.info(f"Starting Portfolio Demo with DUMMY DATA")
    logger.info("Credentials: admin/admin123 or qa/qa123")
    app.run_server(debug=False)

INFO:__main__:Starting Portfolio Demo with DUMMY DATA
INFO:__main__:Credentials: admin/admin123 or qa/qa123
